# Conceptual Analysis Workbook for Large LLMs.
### Plan:
* Same as small with huge models. run at own risk. Large amounts of cache space and memory required.

In [1]:
import os
import torch
import openai
import numpy as np

import sentencepiece    # necessary for proper t5 init.
from transformers import T5Tokenizer, T5EncoderModel, GPT2Tokenizer, OPTModel

from sklearn.metrics.pairwise import cosine_similarity

# api key set in conda env.
openai.api_key = os.getenv('OPENAI_API_KEY')

## Load Models

### Vocab

In [2]:
vocab = []
with open('./vocab/expanded_vocab.txt', 'r') as f:
    for line in f:
        vocab.append(line.strip())

len(vocab)

5124

### GPT-3

In [3]:
# Loading saved embeddings from GPT-3 Curie or DaVinci
gpt_embeds = []
with open(u'./gpt/gpt_curie.txt', 'r') as f:
    for line in f:
        gpt_embeds.append([float(x) for x in line.strip().split()])

model_gpt = dict(zip(vocab, gpt_embeds))

In [4]:
# for getting new embeddings from OpenAI
def gpt_embed(text, engine='text-similarity-curie-001'):
    text = text.replace('\n', ' ')
    return openai.Embedding.create(input=[text], engine=engine)['data'][0]['embedding']

### OPT

In [5]:
# Loding OPT with HuggingFace Transformers. Paramenter size can be changed- ['125m', '350m', '1.3b', '2.7b', '6.7b', '13b', '30b']
# NOTE: 1.3b uses 2.6G of .cache space.

# NOTE: The documentation recommends using the GPT2Tokenizer here.
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-13b', cache_dir='/scratch/mbarlow6/.cache')
model_opt_raw = OPTModel.from_pretrained('facebook/opt-13b', cache_dir='/scratch/mbarlow6/.cache')

Some weights of the model checkpoint at facebook/opt-13b were not used when initializing OPTModel: ['decoder.final_layer_norm.weight', 'decoder.final_layer_norm.bias']
- This IS expected if you are initializing OPTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing OPTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
opt_embeds = []
with open(u'./opt/13B.txt', 'r') as f:
    for line in f:
        opt_embeds.append([float(x) for x in line.strip().split()])
model_opt = dict(zip(vocab, opt_embeds))

In [7]:
# This is how I'm getting the embedding from OPT.
def opt_embed(word, tokenizer=gpt2_tokenizer, model=model_opt_raw, debug=False):
    # prepare inputs and model
    inputs = tokenizer(word, return_tensors="pt") # return pytorch tensors
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)

    # get embedding from final layer
    last_hidden_states = outputs.last_hidden_state # I have confirmed this is the same
                                                   # As the strategy from Tianyi

    # remove unnecessary dimension
    embeddings = torch.squeeze(last_hidden_states, dim=0)

    # add all of the vectors together
    final_embed = np.array(embeddings[1])
    if embeddings.shape[0] > 2:
        for next_emb in embeddings[2:]:
            final_embed += np.array(next_emb)
        final_embed /= embeddings.shape[0] - 1
    return final_embed
    # same outcome as strategy in negatives.ipynb

### T5
Encoder-only mean pooling embeddings most likely replicate the behavior we want - see negatives.ipynb loading section for full comment.

In [8]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-3b', cache_dir='/scratch/mbarlow6/.cache')
model_t5_raw = T5EncoderModel.from_pretrained('t5-3b', cache_dir='/scratch/mbarlow6/.cache')

/home/mbarlow6/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Some weights of the model checkpoint at t5-3b were not used when initializing T5EncoderModel: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.23.layer.1.layer_norm.weight', 'decoder.block.6.layer.2.DenseReluDense.wo.weight

In [9]:
t5_embeds = []
with open('./t5/t53b.txt', 'r') as f:
    for line in f:
        t5_embeds.append([float(x) for x in line.strip().split()])
model_t5 = dict(zip(vocab, t5_embeds))

In [10]:
def t5_embed(text, tokenizer=t5_tokenizer, model=model_t5_raw, debug=False):
    inputs = tokenizer(text, return_tensors='pt')
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)
    # get final hidden state, remove extra dim
    embeddings = torch.squeeze(outputs.last_hidden_state, dim=0)
    # average the embedding of each token by dimension, return.
    return np.array(torch.mean(embeddings, dim=0))

In [11]:
# from tqdm.notebook import tqdm
# for i in tqdm(range(len(vocab))):
#     word = vocab[i]
#     em = t5_embed(word)
#     with open('./t5/t5large.txt', 'a') as f:
#         for p in em:
#             f.write(str(p) + ' ')
#         f.write('\n')

## Define Helpers (for angle)

In [12]:
def positive(words, model='gpt'):
    """
    Args:
        words: iterable
        model: 'gpt', 'opt', or 't5'
    Returns:
        Positive (summed vectors) of word embeddings of a given list of words from the specified model. Defaults to GPT-3.
    """
    if isinstance(words, str):
        print(f"You requested the positive of the string \"{words}\". Did you mean [\"{words}\"]?")

    out = 0
    for token in words:
        # convert token to string
        word = str(token)
        flag = word in vocab
        # do model check - least intensive operation to repeat
        if model.lower() == 'gpt':
            # look for token in cached GPT embeds
            if flag:
                ex = model_gpt[word]  # ex for "extracted"
            # if not found, query API
            else:
                ex = gpt_embed(word)
                model_gpt[word] = ex
        elif model.lower() == 'opt':
            if flag:
                ex = model_opt[word]
            else:
                ex = opt_embed(word)
                model_opt[word] = ex
        elif model.lower() == 't5':
            if flag:
                ex = model_t5[word]
            else:
                ex = t5_embed(word)
                model_t5[word] = ex
        else:
            raise ValueError('Please provide either gpt, opt, or t5 as a model choice.')

        # construct positive
        if isinstance(out, int):
            out = np.array(ex).reshape(1, -1)
        else:
            out += np.array(ex).reshape(1, -1)
            
    return out if not isinstance(out, int) else np.array([])

In [13]:
def most_similar(target, phrase=False, topn=10, model='gpt'):
    """
    Args:
        target: str or list[float] (if phrase)
        phrase: bool    -> True if target is vector. Default False.
        topn: int       -> Number of results to keep [0:2+topn]
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        Top n most similar words in common vocab to the target vector in the specified model.
    """
    # holder for all similarity scores
    sims = {}

    # initialize starting point
    a = target

    # if no phrase, must convert to embed.
    if not phrase:
        a = positive([a], model)

    # now, we check every B in our vocab
    for term in vocab:
        b = positive([term], model)# plural; removal?

        if isinstance(target, str):
            if term != target:
                sims[term] = cosine_similarity(a, b)[0][0]
        elif b.any():
            sims[term] = cosine_similarity(a, b)[0][0]
    
    return list({k: v for k, v in sorted(sims.items(), key=lambda x: x[1], reverse=True)}.items())[:2+topn]

In [14]:
def definition(words, topn=10, model='gpt'):
    """
    Args:
        words: iterable -> Items to be combined into a positive and defined
        topn: int       -> number of words to keep in definiton
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        Top N most similar words in the specified model to a positive of the string representations
        of any provided interable, if it's members can be cast to strings. 
    """
    # get phrase
    if isinstance(words, str):
        words = [words]
    phrase = positive(words, model)

    # get definition
    def_of_positive = most_similar(phrase, True, topn, model)

    return def_of_positive

In [15]:
def define(words, topn=10):
    """
    Args:
        words: string or iterable -> Items to be combined into a positive and defined
        topn: int                 -> number of words to keep in definiton
    Returns:
        None. Formats and prints definition from both models.
    """
    print(*words, sep=', ')
    print('GPT-3:')
    print(definition(words, topn, 'gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(definition(words, topn, 'opt'))
    print('------------------------------------------------------')
    print('T5:')
    print(definition(words, topn, 't5'))
    print('------------------------------------------------------')


In [16]:
def calculate_similarity(words, target, vec=False, model='gpt'):
    """
    Args:
        words: iterable or vector   -> Items to be combined into a positive and compared,
                                       or already constructed vector of matching dimensions.
        target: str     -> single term to calulate similarity to.
        vec: bool       -> true if 'words' is a vector
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        The cosine similarity of the words vector and target term in specified model.
    """
    # get phrase
    phrase = words
    if not vec:
        if isinstance(words, str):
            phrase = positive([words], model)
        else:
            phrase = positive(words, model)
    
    # get target
    target = positive([target], model)

    return cosine_similarity(phrase, target)[0][0]


In [17]:
def sim_test(words, target):
    """
    Args:
        words: iterable, str, or vector -> Items to be combined into a positive and compared,
                                            or already constructed vector of matching dimensions.
        target: str                     -> single term to calulate similarity to.
    Returns:
        None. Formats and prints similarity from both models.
    """
    print(*words, sep=', ', end='')
    print(f' -> {target}')
    print('GPT-3:')
    print(calculate_similarity(words, target, model='gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(calculate_similarity(words, target, model='opt'))
    print('------------------------------------------------------')
    print('T5:')
    print(calculate_similarity(words, target, model='t5'))
    print('------------------------------------------------------')

I'm choosing to leave out a function where both A and B are vectors, as that is mostly helpful for phrase vectors we're not constucting ourselves - so I'll include it later if needed.

## Conduct Angle tests

### A - Bachelors, Husbands, and Wives
Words describing spousal relations.

In [18]:
define(['man', 'unmarried'])

man, unmarried
GPT-3:
[('unmarried', 0.9405478317212387), ('man', 0.9405478314044396), ('single', 0.9044411741901922), ('male', 0.9008872376290222), ('person', 0.898244206756172), ('alone', 0.8953741094447343), ('guy', 0.886661487602376), ('adult', 0.8858356898372889), ('individual', 0.8840143177710547), ('independent', 0.8784498302991113), ('human', 0.8772293208729568), ('off', 0.8749977963152205)]
------------------------------------------------------
OPT:
[('man', 0.9208742827571141), ('kid', 0.9103229513177569), ('leader', 0.9087618850848679), ('brother', 0.9061688421084071), ('girl', 0.9059652487086537), ('mother', 0.9054211399021482), ('fan', 0.9039426508598443), ('woman', 0.9026509613888662), ('worker', 0.9023072310673633), ('father', 0.9022305301645894), ('dog', 0.901959478450079), ('wife', 0.9006638735253008)]
------------------------------------------------------
T5:
[('unmarried', 0.7814224098704085), ('man', 0.7413537044716513), ('monk', 0.5578316745339128), ('woman', 0.553

In [19]:
define(['man', 'married'])

man, married
GPT-3:
[('man', 0.9422816587023249), ('married', 0.9422816551305726), ('husband', 0.9114781468028443), ('male', 0.9012788705025968), ('person', 0.8997014868740365), ('marriage', 0.8960741118000042), ('guy', 0.8933628385523915), ('spouse', 0.892420564601974), ('wife', 0.8904628061543763), ('adult', 0.890378294323545), ('mark', 0.8844042580423119), ('marry', 0.8840830220237552)]
------------------------------------------------------
OPT:
[('runner', 0.9793785063513771), ('married', 0.9790422907596306), ('mate', 0.9787278185510063), ('winner', 0.9782857850871801), ('man', 0.9774070981199011), ('leader', 0.9760694154505745), ('worth', 0.9758737812873503), ('wife', 0.9758698169976546), ('rich', 0.9746450322480772), ('aunt', 0.9742010801709623), ('old', 0.9735639330200521), ('bye', 0.9723407332279532)]
------------------------------------------------------
T5:
[('married', 0.8792515018183467), ('man', 0.8614237876038489), ('husband', 0.8147622153972165), ('woman', 0.801617566464

In [20]:
sim_test(['man', 'unmarried'], 'bachelor')

man, unmarried -> bachelor
GPT-3:
0.865110343915183
------------------------------------------------------
OPT:
0.5491606132226298
------------------------------------------------------
T5:
0.3978905732207702
------------------------------------------------------


In [21]:
sim_test(['man', 'married'], 'husband')

man, married -> husband
GPT-3:
0.9114781468028443
------------------------------------------------------
OPT:
0.9688865470685613
------------------------------------------------------
T5:
0.8147622153972165
------------------------------------------------------


In [18]:
define(['male', 'married', 'person'])

male, married, person
GPT-3:
[('male', 0.9521662646663593), ('person', 0.9499068306000253), ('married', 0.9241847365769442), ('adult', 0.9167241992732257), ('human', 0.9138888889924761), ('guy', 0.911505949894593), ('husband', 0.909733814692772), ('man', 0.9050365362510902), ('spouse', 0.9044276694548496), ('someone', 0.9040077211454194), ('individual', 0.8999292849386467), ('user', 0.8998707926060661)]
------------------------------------------------------
OPT:
[('male', 0.9856864930984773), ('female', 0.9797447911320867), ('worker', 0.9760868977575332), ('aggressive', 0.9754091840142143), ('leader', 0.9735707764247317), ('powerful', 0.9735300495645132), ('woman', 0.9730974992402619), ('qualified', 0.9730452387921813), ('daughter', 0.9723544905936321), ('student', 0.9717298330332856), ('agent', 0.9715698416794665), ('wife', 0.9714953375244051)]
------------------------------------------------------
T5:
[('married', 0.8791610610431412), ('male', 0.8685982925508514), ('female', 0.809435

In [19]:
sim_test(['male', 'married', 'person'], 'husband')

male, married, person -> husband
GPT-3:
0.909733814692772
------------------------------------------------------
OPT:
0.9496257926635936
------------------------------------------------------
T5:
0.7859480355132374
------------------------------------------------------


In [20]:
define(['male', 'married', 'human'])

male, married, human
GPT-3:
[('male', 0.9516948730311056), ('human', 0.9464518955005357), ('married', 0.9229094263881856), ('person', 0.9207804062456757), ('adult', 0.9134503990495724), ('husband', 0.9045016264745342), ('spouse', 0.8985750833935761), ('female', 0.896558565028355), ('individual', 0.895890982171123), ('man', 0.8956539325811075), ('gender', 0.8933205464973085), ('guy', 0.8923694092147755)]
------------------------------------------------------
OPT:
[('male', 0.9882104145213777), ('female', 0.9822433860106375), ('powerful', 0.9817119488504372), ('aggressive', 0.9811424258909969), ('productive', 0.9757099100325038), ('traditional', 0.9749686867197356), ('our', 0.9740799175962455), ('less', 0.9740350944557217), ('adult', 0.9739574599136978), ('qualified', 0.9739092285179096), ('able', 0.9731056270841358), ('ordinary', 0.9730947996301202)]
------------------------------------------------------
T5:
[('married', 0.8550116307372154), ('male', 0.8442788976701776), ('human', 0.814

In [21]:
sim_test(['male', 'married', 'human'], 'husband')

male, married, human -> husband
GPT-3:
0.9045016264745342
------------------------------------------------------
OPT:
0.9249211918690915
------------------------------------------------------
T5:
0.7629158100898408
------------------------------------------------------


In [22]:
define(['woman', 'married'])

woman, married
GPT-3:
[('woman', 0.9508869376239817), ('married', 0.9508869327618845), ('wife', 0.9312897038855579), ('female', 0.9175481487008541), ('spouse', 0.9122517044511196), ('marriage', 0.8974979485732754), ('person', 0.8959440450893408), ('Mrs', 0.8955020549626074), ('lady', 0.8952619249495897), ('girlfriend', 0.8934626139206588), ('husband', 0.8919429545356903), ('mother', 0.8867276809844507)]
------------------------------------------------------
OPT:
[('mate', 0.9827150014667141), ('woman', 0.9817632491642048), ('married', 0.9807480245410097), ('wife', 0.9805473958787398), ('runner', 0.9781956260109116), ('winner', 0.9776849675868977), ('leader', 0.9744230360504296), ('daughter', 0.9738451292364672), ('husband', 0.9728747425350734), ('uncle', 0.9723422887920337), ('worker', 0.9714661961886001), ('brother', 0.9711569909575104)]
------------------------------------------------------
T5:
[('married', 0.935969449089613), ('woman', 0.9127262949313386), ('wife', 0.87313322597273)

In [23]:
sim_test(['woman', 'married'], 'wife')

woman, married -> wife
GPT-3:
0.9312897038855579
------------------------------------------------------
OPT:
0.9805473958787398
------------------------------------------------------
T5:
0.87313322597273
------------------------------------------------------


In [24]:
define(['woman', 'spouse'])

woman, spouse
GPT-3:
[('woman', 0.9592239525768028), ('spouse', 0.9592239497442718), ('wife', 0.9490044420671463), ('female', 0.9243046825121588), ('person', 0.9077347637223891), ('lady', 0.9058937754158713), ('Mrs', 0.9057586971564326), ('husband', 0.90336984468333), ('girlfriend', 0.9024640478305104), ('spouses', 0.8993576788077576), ('partner', 0.8974414786977944), ('mother', 0.8970153688670363)]
------------------------------------------------------
OPT:
[('woman', 0.9032537510846648), ('girlfriend', 0.8998056374355402), ('wife', 0.8981243912506017), ('daughter', 0.8965439728707655), ('husband', 0.8959890067819172), ('worker', 0.8955411267929279), ('girl', 0.893431769542282), ('leader', 0.8920113205779735), ('writer', 0.8897022790231389), ('father', 0.8892330082301777), ('friend', 0.888973779416284), ('mother', 0.8861240981485837)]
------------------------------------------------------
T5:
[('spouse', 0.9026179749591232), ('woman', 0.8661095153369437), ('wife', 0.8522384624466446),

In [25]:
define(['women', 'spouses'])

women, spouses
GPT-3:
[('spouses', 0.9598310883561074), ('wives', 0.942986897262519), ('wife', 0.9084563418848068), ('spouse', 0.9078221246869269), ('people', 0.8979651482468473), ('husbands', 0.8960789319777525), ('girls', 0.8959793122930186), ('woman', 0.8939359268890458), ('couple', 0.8875131142202912), ('female', 0.8857644215703531), ('married', 0.8816482185854636), ('sisters', 0.881278917146149)]
------------------------------------------------------
OPT:
[('people', 0.8965786483074476), ('girls', 0.8922228016011526), ('children', 0.8904762740269638), ('boys', 0.8838598078932913), ('hers', 0.8837565167642195), ('worker', 0.8771524090488501), ('girlfriend', 0.8757027399498993), ('father', 0.8705810091428009), ('wives', 0.8705245501348408), ('pants', 0.8677475232321419), ('friend', 0.8674243385478452), ('shirt', 0.8667876647338009)]
------------------------------------------------------
T5:
[('spouses', 0.8448476893850763), ('woman', 0.7715655728366541), ('wives', 0.7515713174596931

In [26]:
sim_test(['woman', 'spouse'], 'wife')

woman, spouse -> wife
GPT-3:
0.9490044420671463
------------------------------------------------------
OPT:
0.8981243912506017
------------------------------------------------------
T5:
0.8522384624466446
------------------------------------------------------


In [27]:
sim_test(['women', 'spouses'], 'wives')

women, spouses -> wives
GPT-3:
0.942986897262519
------------------------------------------------------
OPT:
0.8705245501348408
------------------------------------------------------
T5:
0.7515713174596931
------------------------------------------------------


In [28]:
define(['female', 'spouse'])

female, spouse
GPT-3:
[('female', 0.957281462145671), ('spouse', 0.9572814593197987), ('wife', 0.9389518874523621), ('woman', 0.9300691797234111), ('girlfriend', 0.9089975947454579), ('spouses', 0.9013469279396742), ('partner', 0.9001715583416265), ('lady', 0.8980371114445169), ('Mrs', 0.8963367323192756), ('person', 0.8925706404221178), ('daughter', 0.8894180147603357), ('married', 0.8875078285546344)]
------------------------------------------------------
OPT:
[('male', 0.9040058563296046), ('female', 0.9029112634134298), ('aggressive', 0.9001660691112114), ('racist', 0.8951813696674574), ('conservative', 0.8911063767954271), ('worker', 0.8905473669709825), ('professional', 0.8863138539525367), ('shirt', 0.8860765102729844), ('artist', 0.8857832708624895), ('girl', 0.8845499530786339), ('liberal', 0.8835120055726873), ('involved', 0.8827760050218514)]
------------------------------------------------------
T5:
[('spouse', 0.8979356902354297), ('female', 0.887471074961633), ('male', 0.

In [29]:
sim_test(['female', 'spouse'], 'wife')

female, spouse -> wife
GPT-3:
0.9389518874523621
------------------------------------------------------
OPT:
0.8791753274501799
------------------------------------------------------
T5:
0.7661055210211959
------------------------------------------------------


In [22]:
define(['female', 'married', 'person'])

female, married, person
GPT-3:
[('female', 0.9479292334237757), ('person', 0.9473210486854957), ('woman', 0.9397273031815065), ('married', 0.9246213729572954), ('wife', 0.9146424703365014), ('human', 0.911782425492916), ('spouse', 0.9106120294931515), ('adult', 0.9081852892007541), ('someone', 0.9053972823252454), ('male', 0.9049956911314825), ('girl', 0.9030108795469753), ('lady', 0.9016945940772227)]
------------------------------------------------------
OPT:
[('female', 0.98488191133383), ('male', 0.982925953786119), ('qualified', 0.977319736917273), ('powerful', 0.9770270285326365), ('aggressive', 0.9768074147301735), ('independent', 0.9747533607969084), ('worker', 0.9737689397851994), ('productive', 0.9732855599914226), ('enemy', 0.9732068659868931), ('state', 0.972646763425189), ('artist', 0.9724972991735281), ('runner', 0.9724026428699633)]
------------------------------------------------------
T5:
[('married', 0.8800818876080156), ('female', 0.8789617665891625), ('woman', 0.810

In [23]:
sim_test(['female', 'married', 'person'], 'wife')

female, married, person -> wife
GPT-3:
0.9146424703365014
------------------------------------------------------
OPT:
0.9697013421782116
------------------------------------------------------
T5:
0.7863141521667425
------------------------------------------------------


In [24]:
define(['female', 'married', 'human'])

female, married, human
GPT-3:
[('female', 0.9478256336853672), ('human', 0.9442308902781799), ('woman', 0.9307560862743403), ('married', 0.9231635630246703), ('person', 0.917944368303415), ('wife', 0.909437820800914), ('adult', 0.9047114738265493), ('spouse', 0.904580812046557), ('male', 0.9042593382088), ('gender', 0.8955730423688701), ('individual', 0.8929550547959717), ('girl', 0.8917971332920928)]
------------------------------------------------------
OPT:
[('female', 0.9872808389128164), ('male', 0.9854119943255649), ('powerful', 0.9850333297912601), ('aggressive', 0.9824199214727224), ('productive', 0.9783736815610413), ('qualified', 0.9781165982141764), ('able', 0.9768347861722539), ('traditional', 0.975867517924031), ('ordinary', 0.9754919029216673), ('less', 0.9747689153529547), ('smoking', 0.9738119765226014), ('active', 0.9737554934101899)]
------------------------------------------------------
T5:
[('married', 0.858406933612684), ('female', 0.8517786215886249), ('human', 0.

In [25]:
sim_test(['female', 'married', 'human'], 'wife')

female, married, human -> wife
GPT-3:
0.909437820800914
------------------------------------------------------
OPT:
0.952957112293022
------------------------------------------------------
T5:
0.7650649862344225
------------------------------------------------------


In [30]:
define(['man', 'spouse'])

man, spouse
GPT-3:
[('man', 0.9452903740221108), ('spouse', 0.945290372245649), ('husband', 0.928038889221394), ('person', 0.9167041917861729), ('wife', 0.9138179884162033), ('male', 0.9047190239266842), ('guy', 0.9004717577011105), ('mate', 0.8982793982351434), ('partner', 0.8978381178121089), ('woman', 0.8937212769510919), ('adult', 0.8890401872005786), ('human', 0.8889923358967983)]
------------------------------------------------------
OPT:
[('guy', 0.8959201766604912), ('man', 0.8939409520894572), ('girlfriend', 0.8878497413500783), ('girl', 0.885822004227153), ('friend', 0.8825239875811781), ('boy', 0.8822261702591676), ('worker', 0.8814487952430528), ('doctor', 0.8802756301847715), ('kid', 0.8800042197732697), ('leader', 0.879916886338442), ('mother', 0.8793740651360951), ('father', 0.8782559731050542)]
------------------------------------------------------
T5:
[('spouse', 0.828477365314363), ('man', 0.8018645787591304), ('husband', 0.7779906157508354), ('wife', 0.76870958521389

In [31]:
define(['men', 'spouses'])

men, spouses
GPT-3:
[('spouses', 0.9495941931344591), ('husbands', 0.9283741995407647), ('wives', 0.9154644730285089), ('spouse', 0.9002873896480527), ('people', 0.8977115495675744), ('couple', 0.8920417795143372), ('husband', 0.8896360235343407), ('them', 0.8889909432855655), ('boys', 0.8834282214832077), ('male', 0.880232553946837), ('wife', 0.87957183559485), ('married', 0.8780621942073294)]
------------------------------------------------------
OPT:
[('hers', 0.8754836643277523), ('girls', 0.8719224176883686), ('children', 0.8711393721614191), ('people', 0.8687218612074341), ('guy', 0.868411754859383), ('boys', 0.8637093253995742), ('man', 0.8622730719590054), ('worker', 0.8608624406894885), ('guard', 0.8605650069602506), ('girlfriend', 0.8587616760405608), ('doctor', 0.8577524732823146), ('rider', 0.8577078723617935)]
------------------------------------------------------
T5:
[('spouses', 0.8125651449239583), ('husband', 0.7258598857825145), ('wives', 0.703154511159204), ('woman',

In [32]:
define(['male', 'spouse'])

male, spouse
GPT-3:
[('spouse', 0.9542306649666342), ('male', 0.9542306640633107), ('husband', 0.9302557338349129), ('wife', 0.909489502502815), ('partner', 0.9030825735646489), ('person', 0.9024235760070216), ('spouses', 0.8962547794330646), ('adult', 0.8937270205526132), ('boyfriend', 0.8929627972100053), ('married', 0.8927926925154542), ('guy', 0.8898254902112422), ('parent', 0.8887843892958963)]
------------------------------------------------------
OPT:
[('male', 0.8984984674108248), ('aggressive', 0.8855784972708444), ('worker', 0.8840399761827351), ('professional', 0.8799651089501739), ('female', 0.8798384666162751), ('girl', 0.8786121701366589), ('adult', 0.8769079031275543), ('student', 0.8757358849253722), ('racist', 0.8757162269567609), ('dog', 0.8749244022491323), ('shirt', 0.8748636485844221), ('girlfriend', 0.8738008994041945)]
------------------------------------------------------
T5:
[('spouse', 0.892346915785927), ('male', 0.862451690924481), ('female', 0.8113878374351

In [33]:
define(['male', 'spouses'])

male, spouses
GPT-3:
[('spouses', 0.9422589859002489), ('male', 0.9422589823999724), ('spouse', 0.9317374276738452), ('husband', 0.9131668820446905), ('husbands', 0.9058325749235947), ('couple', 0.9020221017344937), ('married', 0.8999152032903182), ('wife', 0.894150905870805), ('partner', 0.8876246649129814), ('wives', 0.8873414705846329), ('person', 0.887062458434746), ('adult', 0.8841608960224063)]
------------------------------------------------------
OPT:
[('male', 0.8793861461438286), ('aggressive', 0.8667081359096347), ('worker', 0.8660145426227115), ('female', 0.8617815864524777), ('professional', 0.8609200536954146), ('racist', 0.8598219822724147), ('shirt', 0.8582380190545154), ('police', 0.8575945489120786), ('student', 0.8568902783664627), ('girl', 0.8566995125050272), ('girlfriend', 0.8556905903108741), ('adult', 0.8552170228719648)]
------------------------------------------------------
T5:
[('male', 0.8370362503253069), ('spouses', 0.8194169110910494), ('female', 0.781304

In [34]:
sim_test(['male', 'spouse'], 'husband')

male, spouse -> husband
GPT-3:
0.9302557338349129
------------------------------------------------------
OPT:
0.8499174337154989
------------------------------------------------------
T5:
0.7497094347297405
------------------------------------------------------


In [35]:
sim_test(['male', 'spouses'], 'husbands')

male, spouses -> husbands
GPT-3:
0.9058325749235947
------------------------------------------------------
OPT:
0.5571409149876979
------------------------------------------------------
T5:
0.6460742526449045
------------------------------------------------------


In [36]:
sim_test(['man', 'spouse'], 'husband')

man, spouse -> husband
GPT-3:
0.928038889221394
------------------------------------------------------
OPT:
0.8751676094053835
------------------------------------------------------
T5:
0.7779906157508354
------------------------------------------------------


In [37]:
sim_test(['men', 'spouses'], 'husbands')

men, spouses -> husbands
GPT-3:
0.9283741995407647
------------------------------------------------------
OPT:
0.5769258086429908
------------------------------------------------------
T5:
0.6847319245789221
------------------------------------------------------


### B - Brothers, sisters, and daughters
Family words related to children.

* Male, sibling, brother.

In [38]:
define(['man', 'sibling'])

man, sibling
GPT-3:
[('man', 0.9379309046076323), ('sibling', 0.9379308998850917), ('brother', 0.9233521474784272), ('person', 0.9079277144444915), ('male', 0.9063315152370417), ('siblings', 0.8965444352859131), ('guy', 0.8934967555874964), ('mate', 0.890856755113611), ('adult', 0.8898909830357895), ('sister', 0.8890895796909183), ('human', 0.8872460543096847), ('father', 0.8849263387173191)]
------------------------------------------------------
OPT:
[('man', 0.9272953999359421), ('son', 0.9171465383038662), ('us', 0.912895636606972), ('guy', 0.9128533767787824), ('rider', 0.9126278315764886), ('hall', 0.9116605347335357), ('mother', 0.9091549076517167), ('father', 0.9081417703960325), ('leader', 0.9076115209193157), ('doctor', 0.9072389980812179), ('dog', 0.9063341606330726), ('kid', 0.9052815197730077)]
------------------------------------------------------
T5:
[('sibling', 0.8045590281276656), ('man', 0.7198390292158241), ('son', 0.5489286506655673), ('brother', 0.5388708359165), (

In [39]:
sim_test(['man', 'sibling'], 'brother')

man, sibling -> brother
GPT-3:
0.9233521474784272
------------------------------------------------------
OPT:
0.9001067004841522
------------------------------------------------------
T5:
0.5388708359165
------------------------------------------------------


In [40]:
sim_test(['men', 'siblings'], 'brothers')

men, siblings -> brothers
GPT-3:
0.9327448203600848
------------------------------------------------------
OPT:
0.762997348282519
------------------------------------------------------
T5:
0.7535028671932406
------------------------------------------------------


In [41]:
define(['boy', 'sibling'])

boys, siblings
GPT-3:
[('siblings', 0.95354586565088), ('boys', 0.953545864061933), ('brothers', 0.9404598978232492), ('children', 0.918718904056889), ('sibling', 0.918468537515963), ('sisters', 0.9005319575797786), ('brother', 0.8945584126081514), ('teens', 0.8925475157148333), ('girls', 0.8883271673127995), ('family', 0.8815364895271713), ('them', 0.8762595782679274), ('twin', 0.8721472029273261)]
------------------------------------------------------
OPT:
[('boys', 0.9221510429257027), ('girls', 0.913376783532729), ('hers', 0.9113134464835514), ('people', 0.904792178100576), ('children', 0.9015576127197118), ('pants', 0.8982333963416739), ('wives', 0.8971943940997371), ('arms', 0.8942033767266299), ('you', 0.8810963991048836), ('terms', 0.8786400415895165), ('offspring', 0.8763523602262323), ('rider', 0.8719832944594273)]
------------------------------------------------------
T5:
[('siblings', 0.8891065643222588), ('boys', 0.8780506746590999), ('sisters', 0.7971029099162623), ('brot

In [42]:
sim_test(['boy', 'sibling'], 'brother')

boy, sibling -> brother
GPT-3:
0.9233876267883299
------------------------------------------------------
OPT:
0.865760603723906
------------------------------------------------------
T5:
0.600468420510003
------------------------------------------------------


In [43]:
sim_test(['boys', 'siblings'], 'brothers')

boys, siblings -> brothers
GPT-3:
0.9404598978232492
------------------------------------------------------
OPT:
0.7446792845705155
------------------------------------------------------
T5:
0.7873481899839188
------------------------------------------------------


In [44]:
define(['male', 'sibling'])

male, sibling
GPT-3:
[('male', 0.9515305112489758), ('sibling', 0.9515305101149001), ('brother', 0.9232529436145182), ('siblings', 0.9085178748470271), ('adult', 0.8902256329093771), ('person', 0.8892432262941439), ('gender', 0.884635071918882), ('parent', 0.8843181691356145), ('brothers', 0.8835402569224253), ('buddy', 0.8822795665716101), ('human', 0.8819381229845606), ('sister', 0.8812984871916825)]
------------------------------------------------------
OPT:
[('male', 0.9295743713798768), ('female', 0.9186146067974716), ('student', 0.9140732473648433), ('adult', 0.9082807642004185), ('aggressive', 0.9077565147705532), ('usual', 0.9073334465898347), ('worker', 0.9070884428511786), ('artist', 0.907066059000176), ('player', 0.9070414389309139), ('union', 0.9062735441829792), ('shirt', 0.9057610490987125), ('monster', 0.9056962920756725)]
------------------------------------------------------
T5:
[('sibling', 0.8152132800144124), ('male', 0.7026884750757443), ('female', 0.61567588086204

In [45]:
sim_test(['male', 'sibling'], 'brother')

male, sibling -> brother
GPT-3:
0.9232529436145182
------------------------------------------------------
OPT:
0.8907175623997353
------------------------------------------------------
T5:
0.522321492406666
------------------------------------------------------


In [46]:
sim_test(['male', 'siblings'], 'brothers')

male, siblings -> brothers
GPT-3:
0.9129030885814073
------------------------------------------------------
OPT:
0.7793870061428664
------------------------------------------------------
T5:
0.6968057465898798
------------------------------------------------------


* Female, sibling, sister

In [47]:
define(['female', 'sibling'])

female, sibling
GPT-3:
[('female', 0.9506658672093551), ('sibling', 0.9506658618038598), ('sister', 0.9318368084836333), ('siblings', 0.9111563412985206), ('daughter', 0.9033838532045636), ('woman', 0.9019015915334831), ('sisters', 0.8940712148774717), ('girl', 0.8929024212698267), ('brother', 0.8871846719911936), ('gender', 0.8860328024645858), ('person', 0.8830264877336989), ('male', 0.8792389749965277)]
------------------------------------------------------
OPT:
[('female', 0.9334088571617838), ('male', 0.9279904981062268), ('aggressive', 0.9150462800688706), ('player', 0.9144183718991962), ('artist', 0.9142855985446849), ('usual', 0.9133453164111041), ('student', 0.9133049271761728), ('conservative', 0.9129688007303308), ('powerful', 0.9127422055379918), ('gaming', 0.9114255694928888), ('reader', 0.9114032909760804), ('alone', 0.9101872863327177)]
------------------------------------------------------
T5:
[('sibling', 0.7995695426138654), ('female', 0.7248228116753047), ('male', 0.

In [48]:
define(['female', 'siblings'])

female, siblings
GPT-3:
[('female', 0.9424869511862287), ('siblings', 0.9424869505916196), ('sibling', 0.9354921737847339), ('sisters', 0.9233817729008995), ('sister', 0.9181875708776281), ('daughter', 0.9017280367344722), ('woman', 0.8952490825155495), ('girls', 0.8942637296996637), ('family', 0.8883152417633525), ('children', 0.8876250013531835), ('girl', 0.8875433860628836), ('gender', 0.883767621827728)]
------------------------------------------------------
OPT:
[('female', 0.8992811813236332), ('male', 0.8931806035468579), ('girls', 0.8923903451163584), ('boys', 0.8835977310624433), ('shirt', 0.8835969510569818), ('aggressive', 0.8835436861865937), ('hers', 0.8834809990194028), ('people', 0.8834153574248513), ('worker', 0.8834071855662057), ('reader', 0.8827131839704225), ('artist', 0.8823096488742262), ('conservative', 0.8821854756599946)]
------------------------------------------------------
T5:
[('siblings', 0.8712372930614907), ('female', 0.8617695636150093), ('male', 0.7681

In [49]:
sim_test(['female', 'sibling'], 'sister')

female, sibling -> sister
GPT-3:
0.9318368084836333
------------------------------------------------------
OPT:
0.6721912216732029
------------------------------------------------------
T5:
0.5890219741219893
------------------------------------------------------


In [50]:
sim_test(['female', 'siblings'], 'sisters')

female, siblings -> sisters
GPT-3:
0.9233817729008995
------------------------------------------------------
OPT:
0.6575680912531952
------------------------------------------------------
T5:
0.7547792691464682
------------------------------------------------------


In [51]:
define(['woman', 'sibling'])

woman, sibling
GPT-3:
[('woman', 0.9419021533852271), ('sibling', 0.9419021467907442), ('sister', 0.9309789937655348), ('female', 0.9279020982629149), ('person', 0.9085261559711818), ('daughter', 0.9044552294053473), ('siblings', 0.9039758131052908), ('girl', 0.9007231861046295), ('mother', 0.9004904839963023), ('wife', 0.8985409668227056), ('brother', 0.8963072285315794), ('lady', 0.8914530677726235)]
------------------------------------------------------
OPT:
[('woman', 0.9305376600112092), ('wife', 0.9261531668222249), ('leader', 0.9232219688762509), ('daughter', 0.9229616834206098), ('worker', 0.9228538650708557), ('father', 0.9226027825434835), ('man', 0.9210791591646867), ('mother', 0.9196098564400395), ('reader', 0.9190784839897926), ('husband', 0.918755547450289), ('girlfriend', 0.9186592355543756), ('owner', 0.918346186756615)]
------------------------------------------------------
T5:
[('sibling', 0.8268998106694064), ('woman', 0.6987097959316986), ('wife', 0.5905124698905143

In [52]:
define(['women', 'siblings'])

women, siblings
GPT-3:
[('siblings', 0.9491635579872701), ('sisters', 0.9379957914891193), ('sibling', 0.9157841160945128), ('girls', 0.9150322272759045), ('children', 0.9145741018693094), ('people', 0.9021595223054648), ('daughters', 0.8985015857153384), ('wives', 0.897884938935068), ('brothers', 0.8977355626513411), ('family', 0.8930629819669109), ('spouses', 0.8904776887652424), ('sister', 0.8895174615989948)]
------------------------------------------------------
OPT:
[('boys', 0.9104308143966614), ('hers', 0.9092367080701668), ('girls', 0.9074471442354668), ('children', 0.9066336725134458), ('people', 0.9059632657132383), ('wives', 0.8921241028630376), ('arms', 0.8907792661774799), ('pants', 0.8884732709437011), ('worker', 0.8829914086757007), ('father', 0.8815511414924466), ('girlfriend', 0.878804693223089), ('rider', 0.8781657872815128)]
------------------------------------------------------
T5:
[('siblings', 0.886412363406417), ('sisters', 0.7779840664940232), ('wives', 0.75118

In [53]:
sim_test(['woman', 'sibling'], 'sister')

woman, sibling -> sister
GPT-3:
0.9309789937655348
------------------------------------------------------
OPT:
0.6776852212943166
------------------------------------------------------
T5:
0.5885722163193413
------------------------------------------------------


In [54]:
sim_test(['women', 'siblings'], 'sisters')

women, siblings -> sisters
GPT-3:
0.9379957914891193
------------------------------------------------------
OPT:
0.6718164803739315
------------------------------------------------------
T5:
0.7779840664940232
------------------------------------------------------


In [55]:
define(['girl', 'sibling'])

girl, sibling
GPT-3:
[('girl', 0.9439216523190226), ('sibling', 0.9439216478170519), ('sister', 0.9265655458081394), ('female', 0.912818723793685), ('daughter', 0.9096618378752055), ('siblings', 0.9074459402761322), ('girls', 0.8985026666671694), ('woman', 0.8947614285347116), ('child', 0.8932482382519906), ('brother', 0.8927154564964841), ('person', 0.8914720736516533), ('sisters', 0.887608444519628)]
------------------------------------------------------
OPT:
[('girl', 0.9349505988832255), ('boy', 0.9200150131533653), ('monster', 0.9194204198399564), ('girls', 0.9174155217018374), ('mother', 0.91549557558843), ('shirt', 0.9151833644212461), ('reader', 0.9146837416617003), ('guy', 0.9142576249186821), ('girlfriend', 0.9129685222413919), ('dog', 0.9127568957397137), ('sword', 0.9105657064229837), ('beer', 0.9100948764027101)]
------------------------------------------------------
T5:
[('sibling', 0.806821318971906), ('girl', 0.714379830532412), ('girls', 0.616091240558655), ('sisters',

In [56]:
define(['girl', 'siblings'])

girl, siblings
GPT-3:
[('siblings', 0.9373549443356162), ('girl', 0.9373549431184502), ('sibling', 0.9269825356866691), ('girls', 0.9196358457480609), ('sisters', 0.9154964413148391), ('sister', 0.9112019343797898), ('daughter', 0.9064871580630949), ('female', 0.9026947869803514), ('children', 0.898655438744788), ('girlfriend', 0.8901546226845142), ('woman', 0.8864712472724097), ('daughters', 0.8825597482416656)]
------------------------------------------------------
OPT:
[('girl', 0.902623658646718), ('girls', 0.8977260678966288), ('girlfriend', 0.8892139563493284), ('boy', 0.8879375036668536), ('guy', 0.8847963152453988), ('shirt', 0.88380122430734), ('mother', 0.8820111755551102), ('people', 0.8820029898191687), ('monster', 0.8819330118931292), ('reader', 0.8807478645409209), ('boys', 0.8779881219837501), ('children', 0.8776392116087365)]
------------------------------------------------------
T5:
[('siblings', 0.8731924550265341), ('girl', 0.8541194538389142), ('sisters', 0.79627985

In [57]:
sim_test(['girl', 'sibling'], 'sister')

girl, sibling -> sister
GPT-3:
0.9265655458081394
------------------------------------------------------
OPT:
0.7004832253844336
------------------------------------------------------
T5:
0.5923606144753812
------------------------------------------------------


In [58]:
sim_test(['girls', 'siblings'], 'sisters')

girls, siblings -> sisters
GPT-3:
0.9423205163006048
------------------------------------------------------
OPT:
0.6992134852349776
------------------------------------------------------
T5:
0.8142497552414047
------------------------------------------------------


* Female, children, daughter

In [59]:
define(['girl', 'children'])

girl, children
GPT-3:
[('girl', 0.9535646742923878), ('children', 0.9535646723711071), ('girls', 0.936234455218471), ('child', 0.9329542463018093), ('daughter', 0.911279942639336), ('woman', 0.9035184089915791), ('female', 0.9033107484770255), ('kid', 0.9024990792231603), ('baby', 0.8972218798274953), ('boys', 0.8909234909955829), ('youth', 0.890210883135772), ('people', 0.890107354910694)]
------------------------------------------------------
OPT:
[('children', 0.988556407541736), ('girl', 0.9855596315369946), ('girls', 0.9833762857908099), ('mother', 0.9818051750868932), ('father', 0.9795932072730159), ('girlfriend', 0.9789393413909138), ('people', 0.9781444253581744), ('shirt', 0.9771244243854412), ('worker', 0.9765541417437698), ('uncle', 0.9743658233225999), ('boys', 0.9743406516272555), ('wife', 0.974275075598614)]
------------------------------------------------------
T5:
[('girl', 0.894823282424458), ('children', 0.8451943491594494), ('child', 0.796180790536424), ('girls', 0.7

In [60]:
define(['woman', 'children'])

woman, children
GPT-3:
[('woman', 0.9508723453372248), ('children', 0.9508723416815217), ('child', 0.9214348126413545), ('female', 0.9188708138240104), ('girl', 0.9114689425404987), ('daughter', 0.9067707026150459), ('people', 0.9050403855181703), ('girls', 0.9043853066333236), ('person', 0.9027763349839474), ('mother', 0.9009341454900097), ('lady', 0.8958227360670801), ('wife', 0.8914012602232984)]
------------------------------------------------------
OPT:
[('children', 0.9890371708412347), ('woman', 0.9862563265647241), ('father', 0.9856289547068453), ('wife', 0.9856097946716642), ('worker', 0.9820679842813215), ('uncle', 0.9820087446141635), ('winner', 0.9803861096473463), ('owner', 0.9799434685616947), ('leader', 0.979164575947743), ('runner', 0.9789568013278706), ('husband', 0.9772884026395513), ('mother', 0.9762351322993039)]
------------------------------------------------------
T5:
[('woman', 0.8916142171983279), ('children', 0.8632919223993086), ('child', 0.8196297243680738),

In [61]:
define(['female', 'children'])

female, children
GPT-3:
[('female', 0.9505269341769791), ('children', 0.950526930663038), ('woman', 0.9198956690534117), ('girls', 0.9190342341771789), ('child', 0.918789235020015), ('daughter', 0.9143676206048106), ('girl', 0.9122827838692343), ('baby', 0.8938226214147539), ('people', 0.892181607962082), ('youth', 0.8889300976955785), ('gender', 0.8888175391420228), ('lady', 0.8863844956954647)]
------------------------------------------------------
OPT:
[('children', 0.9831109190235724), ('female', 0.979518625466219), ('worker', 0.9793247875673055), ('father', 0.9768129174497324), ('girls', 0.9758069946996859), ('culture', 0.9750546858060176), ('people', 0.9748892101784485), ('wife', 0.974686549185844), ('boys', 0.9737669029932419), ('shirt', 0.972845281164809), ('runner', 0.9721737094793401), ('likely', 0.9719249373686739)]
------------------------------------------------------
T5:
[('female', 0.8872002647802328), ('children', 0.8214105611181435), ('male', 0.7737946196836663), ('chi

In [62]:
sim_test(['girl', 'children'], 'daughter')

girl, children -> daughter
GPT-3:
0.911279942639336
------------------------------------------------------
OPT:
0.9712412523597556
------------------------------------------------------
T5:
0.7347911116102566
------------------------------------------------------


In [63]:
sim_test(['woman', 'children'], 'daughter')

woman, children -> daughter
GPT-3:
0.9067707026150459
------------------------------------------------------
OPT:
0.9759247520743835
------------------------------------------------------
T5:
0.7331171997545636
------------------------------------------------------


In [64]:
sim_test(['female', 'children'], 'daughter')

female, children -> daughter
GPT-3:
0.9143676206048106
------------------------------------------------------
OPT:
0.9679864615695614
------------------------------------------------------
T5:
0.7094045613300592
------------------------------------------------------


In [65]:
define(['female', 'offspring'])

female, offspring
GPT-3:
[('female', 0.9355659353753278), ('offspring', 0.9355659277635209), ('woman', 0.895067117087206), ('daughter', 0.8944931586618226), ('girl', 0.883595902881737), ('gender', 0.8728564170469317), ('girls', 0.8711102438101568), ('source', 0.8705471002840344), ('children', 0.8694956107240368), ('lady', 0.8688987615849869), ('breed', 0.8670119426135573), ('mother', 0.8661357574325835)]
------------------------------------------------------
OPT:
[('female', 0.962069088112496), ('culture', 0.9537743545380045), ('confidence', 0.9500264345767038), ('male', 0.9498927911736067), ('productive', 0.9498774483046478), ('equality', 0.9486757051396997), ('gaming', 0.9483373046584831), ('strength', 0.9482780193385941), ('shirt', 0.9479177531866356), ('alone', 0.9476344049708885), ('aggressive', 0.947486737125064), ('successful', 0.9467216554945322)]
------------------------------------------------------
T5:
[('offspring', 0.8068315257449906), ('female', 0.7512714537054674), ('off

In [66]:
define(['female', 'offsprings'])

female, offsprings
GPT-3:
[('female', 0.9348176762049247), ('offsprings', 0.934817669958265), ('woman', 0.8962693871038516), ('daughter', 0.8924243664737831), ('girl', 0.8845455135435951), ('lady', 0.8750631744250033), ('gender', 0.8711493203664888), ('girls', 0.8694945581242065), ('source', 0.8671888690110401), ('unknown', 0.8666038351358135), ('children', 0.866490260329535), ('youth', 0.8659023014769673)]
------------------------------------------------------
OPT:
[('female', 0.9273980893658847), ('male', 0.9190206519559911), ('culture', 0.9188554343349358), ('shirt', 0.9165505194627119), ('gaming', 0.9153438313277427), ('productive', 0.9149951675671526), ('aggressive', 0.9129890929765879), ('history', 0.9123605386453856), ('strength', 0.9123302419871785), ('alone', 0.9122445128862889), ('usual', 0.9120584859765346), ('powerful', 0.9116887017433355)]
------------------------------------------------------
T5:
[('offsprings', 0.7880654567056797), ('female', 0.7620987803448724), ('offsp

In [67]:
sim_test(['female', 'offspring'], 'daughter')

female, offspring -> daughter
GPT-3:
0.8944931586618226
------------------------------------------------------
OPT:
0.9361831592069447
------------------------------------------------------
T5:
0.5652095139545701
------------------------------------------------------


In [68]:
sim_test(['female', 'offsprings'], 'daughters')

female, offsprings -> daughters
GPT-3:
0.8515757873968931
------------------------------------------------------
OPT:
0.5102705904198142
------------------------------------------------------
T5:
0.6027031968068873
------------------------------------------------------


In [69]:
define(['woman', 'offspring'])

woman, offspring
GPT-3:
[('woman', 0.9312179725606267), ('offspring', 0.9312179641793157), ('female', 0.9079625151993365), ('daughter', 0.8912515510963401), ('girl', 0.8872288824750065), ('mother', 0.8862875187479033), ('person', 0.8861065603326783), ('lady', 0.8829219754692478), ('wife', 0.8747414462456056), ('children', 0.8742606598074814), ('parent', 0.8715884651337653), ('source', 0.8707483268583994)]
------------------------------------------------------
OPT:
[('woman', 0.9621627783342455), ('father', 0.9570497000300109), ('wife', 0.9562050947933364), ('children', 0.9551327355164058), ('worker', 0.9538326947830706), ('daughter', 0.9536152922104147), ('winner', 0.9523513445895821), ('uncle', 0.9518052484945502), ('leader', 0.9516572815318747), ('anger', 0.9504988443430414), ('runner', 0.9504125902806144), ('culture', 0.9503337318334943)]
------------------------------------------------------
T5:
[('offspring', 0.8301445441914582), ('woman', 0.7230817595820576), ('offsprings', 0.664

In [70]:
define(['woman', 'offsprings'])

woman, offsprings
GPT-3:
[('woman', 0.9314617937535629), ('offsprings', 0.9314617869376711), ('female', 0.9062223342087928), ('daughter', 0.8882234480637134), ('lady', 0.8881794808329848), ('girl', 0.8872398639221568), ('person', 0.8849151498640848), ('mother', 0.8843353043442821), ('children', 0.8703171528021552), ('source', 0.8664507423547666), ('wife', 0.8662261257389305), ('random', 0.8657908565837485)]
------------------------------------------------------
OPT:
[('woman', 0.927348750068806), ('father', 0.9231145686209579), ('wife', 0.9211955517017227), ('anger', 0.9183484502872109), ('leader', 0.9182939330882921), ('worker', 0.9182467820295805), ('daughter', 0.917825208611487), ('children', 0.9174427055660092), ('winner', 0.9173981781978777), ('uncle', 0.9158466590437487), ('girlfriend', 0.915540890300923), ('mother', 0.9151798336630547)]
------------------------------------------------------
T5:
[('offsprings', 0.8064623727478903), ('woman', 0.7241080906513634), ('offspring', 0.6

In [71]:
sim_test(['woman', 'offspring'], 'daughter')

woman, offspring -> daughter
GPT-3:
0.8912515510963401
------------------------------------------------------
OPT:
0.9536152922104147
------------------------------------------------------
T5:
0.5918042824786697
------------------------------------------------------


In [72]:
sim_test(['women', 'offsprings'], 'daughters')

women, offsprings -> daughters
GPT-3:
0.8711211905724825
------------------------------------------------------
OPT:
0.5306796146055994
------------------------------------------------------
T5:
0.6108768859089816
------------------------------------------------------


In [73]:
define(['female', 'child'])

female, child
GPT-3:
[('female', 0.9556129211117854), ('child', 0.9556129173203358), ('girl', 0.9285748451137035), ('woman', 0.9274219664788086), ('baby', 0.9187276283206001), ('daughter', 0.9158992633171912), ('person', 0.9066121974158246), ('kid', 0.9050735959395005), ('children', 0.9037543269362093), ('lady', 0.8956424571837502), ('human', 0.8954029149111639), ('infant', 0.8939492230602979)]
------------------------------------------------------
OPT:
[('male', 0.9842862143297129), ('female', 0.9814756101828254), ('student', 0.9789661053872925), ('aggressive', 0.9784298289454063), ('worker', 0.9764364660821218), ('child', 0.9761069300646043), ('culture', 0.9754564117196087), ('nation', 0.975415157612229), ('independent', 0.9742847547386274), ('productive', 0.9735869271981763), ('father', 0.9731757459430112), ('daughter', 0.9727598075707857)]
------------------------------------------------------
T5:
[('female', 0.8820536643144153), ('child', 0.8645593944181609), ('woman', 0.778720209

In [74]:
sim_test(['female', 'child'], 'girl')

female, child -> girl
GPT-3:
0.9285748451137035
------------------------------------------------------
OPT:
0.9605930419469724
------------------------------------------------------
T5:
0.6965056740970761
------------------------------------------------------


In [75]:
# plural; removal?
sim_test(['female', 'children'], 'girls')

female, children -> girls
GPT-3:
0.9190342341771789
------------------------------------------------------
OPT:
0.9758069946996859
------------------------------------------------------
T5:
0.6536111951147099
------------------------------------------------------


In [76]:
define(['male', 'child'])

male, child
GPT-3:
[('child', 0.9552958281925981), ('male', 0.9552958281696193), ('boy', 0.9270367542897525), ('kid', 0.9160068475786277), ('baby', 0.9154808089508516), ('person', 0.913904599217434), ('adult', 0.9044792155050556), ('human', 0.9018637726619886), ('children', 0.8983411190580026), ('parent', 0.8959922908427852), ('infant', 0.8947765134296506), ('guy', 0.8925200232548791)]
------------------------------------------------------
OPT:
[('male', 0.9847420401002751), ('child', 0.9820525176555461), ('student', 0.9790683248366648), ('worker', 0.9763127313689793), ('dog', 0.9737820216461954), ('aggressive', 0.9729168475681622), ('patient', 0.972742032986458), ('monster', 0.9719988437185013), ('professional', 0.9719062798123953), ('father', 0.9707252513324607), ('mother', 0.970651087889422), ('worm', 0.9706225680275216)]
------------------------------------------------------
T5:
[('child', 0.8673988669754498), ('male', 0.8662380497180904), ('female', 0.7906613175352133), ('children

In [77]:
sim_test(['male', 'child'], 'boy')

male, child -> boy
GPT-3:
0.9270367542897525
------------------------------------------------------
OPT:
0.9370228364900198
------------------------------------------------------
T5:
0.6721313666949984
------------------------------------------------------


In [78]:
define(['male', 'offspring'])

male, offspring
GPT-3:
[('male', 0.9366251625335962), ('offspring', 0.9366251605536609), ('adult', 0.8808001152646294), ('parent', 0.872037213443354), ('gender', 0.8712672705366022), ('person', 0.8707407244523123), ('breed', 0.8695386290291609), ('mark', 0.8666018704798473), ('youth', 0.8662633811092489), ('source', 0.8662231076895368), ('boys', 0.8652369845196202), ('human', 0.8650238925992408)]
------------------------------------------------------
OPT:
[('male', 0.9578128266143574), ('female', 0.9561528968613587), ('shirt', 0.9515148530762738), ('culture', 0.9510031087150801), ('worker', 0.9507371592595808), ('productive', 0.9481132157827671), ('aggressive', 0.948036523551963), ('usual', 0.9458674380909432), ('alone', 0.944788125363677), ('student', 0.9445068209187875), ('confidence', 0.9444192673386216), ('gaming', 0.9442210233868907)]
------------------------------------------------------
T5:
[('offspring', 0.8140349364436048), ('male', 0.71912065622792), ('offsprings', 0.66262809

In [79]:
sim_test(['male', 'offspring'], 'son')

male, offspring -> son
GPT-3:
0.858978136867609
------------------------------------------------------
OPT:
0.9029610848939286
------------------------------------------------------
T5:
0.5469945475963789
------------------------------------------------------


In [20]:
define(['male', 'young', 'human'])

male, young, human
GPT-3:
[('male', 0.9531227522419436), ('human', 0.9525693707929181), ('young', 0.9383299703039345), ('person', 0.9229999142294361), ('adult', 0.9060106320533696), ('individual', 0.9043768226254167), ('youth', 0.9015161095947894), ('female', 0.8982727751864357), ('boy', 0.896986108443941), ('guy', 0.8936729103727205), ('gender', 0.8933035718985294), ('regular', 0.8913773576883239)]
------------------------------------------------------
OPT:
[('male', 0.9861027440189138), ('poor', 0.9758442642691804), ('gay', 0.9744399420656228), ('female', 0.9728303804118487), ('traditional', 0.9726223172370455), ('animal', 0.9719843300358523), ('powerful', 0.9705570584003761), ('professional', 0.9694926050923837), ('aggressive', 0.9688686535770199), ('our', 0.9684947767998539), ('adult', 0.9680861296314915), ('healthy', 0.9680798190264233)]
------------------------------------------------------
T5:
[('young', 0.8389448239757569), ('male', 0.827605661768294), ('human', 0.8164013349926

In [21]:
define(['female', 'young', 'human'])

female, young, human
GPT-3:
[('human', 0.9504164244502923), ('female', 0.9491659861684407), ('young', 0.9384915666603033), ('woman', 0.9230488873845349), ('person', 0.9202277052181016), ('girl', 0.9109764281366235), ('male', 0.9061097500329736), ('individual', 0.9015176793303614), ('youth', 0.9001631857237731), ('adult', 0.8973625361233186), ('gender', 0.8955731496928232), ('child', 0.8930135110676246)]
------------------------------------------------------
OPT:
[('male', 0.9859713252163318), ('female', 0.9809382481181761), ('powerful', 0.9769005493331012), ('poor', 0.9763191080679062), ('traditional', 0.97630608613896), ('aggressive', 0.9730979863949359), ('healthy', 0.9715917052004281), ('gay', 0.9711133625660302), ('liberal', 0.9710681666500022), ('usual', 0.970860375839086), ('functional', 0.9701960912368067), ('less', 0.9699777657765268)]
------------------------------------------------------
T5:
[('young', 0.8381667009072654), ('female', 0.832294991736775), ('human', 0.8093489926

### C - Animals, diminutives, and augmentatives!
Is a baby duck a duckling?

In [80]:
define(['baby', 'duck'])

baby, duck
GPT-3:
[('baby', 0.9474069360869881), ('duck', 0.9474069360857743), ('duckling', 0.9138295124498534), ('child', 0.8851431087603283), ('infant', 0.8825326472865318), ('bird', 0.8822564395517654), ('kid', 0.8732957223343283), ('fish', 0.8725115377155359), ('dog', 0.8666608207499897), ('egg', 0.8657717580518123), ('buck', 0.8641755938983218), ('bubble', 0.8637259183542336)]
------------------------------------------------------
OPT:
[('baby', 0.9199118741029286), ('cream', 0.9043886220756834), ('rush', 0.9041795589645918), ('monster', 0.9040783665101813), ('dog', 0.9028235364618333), ('bug', 0.9017980250927529), ('black', 0.9006803972435129), ('meat', 0.9003824632106555), ('dry', 0.9002212479020598), ('dump', 0.8982505129578071), ('mud', 0.8981873990318856), ('kid', 0.8965591587418085)]
------------------------------------------------------
T5:
[('duck', 0.8868582132626475), ('baby', 0.8041941567727864), ('chicken', 0.7212334818986683), ('dad', 0.6889789088566771), ('dog', 0.67

In [81]:
sim_test(['baby', 'duck'], 'duckling')

baby, duck -> duckling
GPT-3:
0.9138295124498534
------------------------------------------------------
OPT:
0.393023787693926
------------------------------------------------------
T5:
0.3149705295258709
------------------------------------------------------


In [82]:
define(['small', 'duck'])

small, duck
GPT-3:
[('duck', 0.9341310534578381), ('small', 0.9341310520990014), ('little', 0.9059533304337686), ('duckling', 0.8979767437929029), ('tiny', 0.8949650667670042), ('short', 0.8720079436736179), ('minor', 0.871763386031372), ('few', 0.8693213768134216), ('low', 0.8663033300324243), ('baby', 0.8662641085910925), ('cute', 0.8620496207174563), ('fish', 0.8602557614249746)]
------------------------------------------------------
OPT:
[('small', 0.9166040018177323), ('large', 0.9049267321876873), ('dry', 0.9034521780172784), ('black', 0.9001640746155352), ('root', 0.8972438526063937), ('colour', 0.8964097456533471), ('smart', 0.8962499119380444), ('rush', 0.895750227291754), ('story', 0.8940450853985937), ('fine', 0.893773726872678), ('grey', 0.8929444991878811), ('race', 0.8928600853131707)]
------------------------------------------------------
T5:
[('duck', 0.8346042670808482), ('small', 0.7331782820897532), ('tiny', 0.639789232004548), ('chicken', 0.6229408851438298), ('blan

In [83]:
sim_test(['small', 'duck'], 'duckling')

small, duck -> duckling
GPT-3:
0.8979767437929029
------------------------------------------------------
OPT:
0.3716206527768686
------------------------------------------------------
T5:
0.30270573247728144
------------------------------------------------------


In [84]:
define(['little', 'duck'])

little, duck
GPT-3:
[('little', 0.9342139046455208), ('duck', 0.9342139003235364), ('small', 0.9057072971711455), ('duckling', 0.9015619547370626), ('tiny', 0.8860890491675141), ('baby', 0.876776232667953), ('few', 0.8718911845929516), ('bit', 0.8662441464590428), ('minor', 0.8650510394220641), ('cute', 0.8649929274095105), ('low', 0.8632402274274141), ('mere', 0.8628348725946339)]
------------------------------------------------------
OPT:
[('little', 0.9159185563457196), ('bare', 0.8979667341761001), ('bag', 0.8970073583366094), ('one', 0.8951265088468848), ('free', 0.8933603029814536), ('baby', 0.8927548586664372), ('rush', 0.892083314674602), ('nice', 0.8912747416242242), ('fine', 0.8909796132394106), ('ice', 0.890824836202782), ('few', 0.8902291846930814), ('cake', 0.8895865926955049)]
------------------------------------------------------
T5:
[('duck', 0.8665865057209516), ('little', 0.7135067169817295), ('feather', 0.6476883730975922), ('chicken', 0.6450600855087232), ('pants', 

In [85]:
sim_test(['little', 'duck'], 'duckling')

little, duck -> duckling
GPT-3:
0.9015619547370626
------------------------------------------------------
OPT:
0.35470416939495186
------------------------------------------------------
T5:
0.3055885790875495
------------------------------------------------------


In [86]:
define(['child', 'duck'])

child, duck
GPT-3:
[('child', 0.9373897225386636), ('duck', 0.9373897217009001), ('baby', 0.9147434754667174), ('duckling', 0.9116574281308549), ('kid', 0.9062932748779746), ('children', 0.893107595934886), ('infant', 0.8809233232756959), ('bird', 0.8802146388706203), ('boy', 0.8787240073076085), ('person', 0.8768617510243462), ('daughter', 0.8746218383692319), ('dog', 0.8740872011934603)]
------------------------------------------------------
OPT:
[('child', 0.9160393911529678), ('dog', 0.9131983510372064), ('bird', 0.8951865369014824), ('friend', 0.8938698706142494), ('monster', 0.8929538769555339), ('shadow', 0.8916772774582327), ('rush', 0.8907587511352435), ('head', 0.8902625018056418), ('master', 0.8896306460062599), ('cycle', 0.88939125521302), ('mother', 0.8885979228597859), ('bag', 0.8880482141745006)]
------------------------------------------------------
T5:
[('duck', 0.8658713965622333), ('child', 0.8129616550810704), ('chicken', 0.7284129575853819), ('children', 0.72294963

In [87]:
sim_test(['child', 'duck'], 'duckling')

child, duck -> duckling
GPT-3:
0.9116574281308549
------------------------------------------------------
OPT:
0.38263256527885525
------------------------------------------------------
T5:
0.3058529825935329
------------------------------------------------------


In [88]:
define(['young', 'duck'])

young, duck
GPT-3:
[('young', 0.9337230212839139), ('duck', 0.9337230157967809), ('duckling', 0.9063798568166693), ('youth', 0.8929357366544689), ('baby', 0.8795233265837883), ('youngster', 0.8778250293596953), ('day', 0.8693624365515091), ('kid', 0.8678037748766143), ('bird', 0.8670383188278186), ('early', 0.866847654243834), ('child', 0.866398025144931), ('random', 0.8638541934184618)]
------------------------------------------------------
OPT:
[('young', 0.9111105453544441), ('gay', 0.886835175262275), ('bad', 0.8805995675498263), ('rush', 0.8744765479212876), ('big', 0.8742857370935555), ('mud', 0.8719614122543757), ('brown', 0.8681195781204123), ('dog', 0.8674007706550775), ('black', 0.8656288800849192), ('boy', 0.8654373649651552), ('dry', 0.8644386194386745), ('poor', 0.8640278851635004)]
------------------------------------------------------
T5:
[('duck', 0.8453132444570348), ('young', 0.8385879877885202), ('lucky', 0.694773993769553), ('boy', 0.674316934165855), ('lump', 0.673

In [89]:
sim_test(['young', 'duck'], 'duckling')

young, duck -> duckling
GPT-3:
0.9063798568166693
------------------------------------------------------
OPT:
0.39118298761134634
------------------------------------------------------
T5:
0.3340382418478141
------------------------------------------------------


In [90]:
define(['small', 'cat'])

small, cat
GPT-3:
[('cat', 0.9363153114339497), ('small', 0.9363153040104342), ('little', 0.9073917454154967), ('pet', 0.8962700309168343), ('kitten', 0.8942536797702618), ('tiny', 0.8941613155834723), ('dog', 0.8809480510389023), ('short', 0.8754289983006616), ('baby', 0.8732492598752075), ('animal', 0.8723022868597468), ('few', 0.8656645784916268), ('minor', 0.8628918601415947)]
------------------------------------------------------
OPT:
[('small', 0.9795048244968394), ('cat', 0.9761644190053385), ('hole', 0.9752600170243617), ('tall', 0.9740416345814732), ('skin', 0.9738656491874984), ('large', 0.9730456854057805), ('jet', 0.9708022594219581), ('belt', 0.9691952105292624), ('eye', 0.9687888140863238), ('tube', 0.9686747357566429), ('string', 0.9686193005211133), ('foot', 0.9685251190517216)]
------------------------------------------------------
T5:
[('cat', 0.8410059597605442), ('small', 0.7590825724856528), ('pet', 0.6468499372599975), ('short', 0.6419635962796449), ('little', 0.6

In [91]:
sim_test(['small', 'cat'], 'kitten')

small, cat -> kitten
GPT-3:
0.8942536797702618
------------------------------------------------------
OPT:
0.5537142726214743
------------------------------------------------------
T5:
0.6123823969121849
------------------------------------------------------


In [92]:
define(['baby', 'cat'])

baby, cat
GPT-3:
[('cat', 0.9518468115873022), ('baby', 0.9518468070064643), ('kitten', 0.9152564146569642), ('pet', 0.9038439072419175), ('dog', 0.9023520149053146), ('child', 0.9010832494033811), ('kid', 0.8905697886127197), ('animal', 0.8905694318753952), ('infant', 0.8871332518396886), ('puppy', 0.8865142305013041), ('boy', 0.8698770396527222), ('bird', 0.8682098342461648)]
------------------------------------------------------
OPT:
[('baby', 0.9816576755555735), ('cat', 0.9778792714197692), ('kid', 0.977565993186656), ('monster', 0.975495276703633), ('dog', 0.9725819069991887), ('rush', 0.9718330271005008), ('skin', 0.9713693895139507), ('egg', 0.9707292074075434), ('bug', 0.9704187193263752), ('hole', 0.9677884341716192), ('eye', 0.9676288879111908), ('beard', 0.9671918988975502)]
------------------------------------------------------
T5:
[('cat', 0.903646814802471), ('baby', 0.8435935803011393), ('pet', 0.7250331928731156), ('egg', 0.7035194889957818), ('fat', 0.7019787691271799

In [93]:
sim_test(['baby', 'cat'], 'kitten')

baby, cat -> kitten
GPT-3:
0.9152564146569642
------------------------------------------------------
OPT:
0.5664929230158181
------------------------------------------------------
T5:
0.6563877417839388
------------------------------------------------------


In [94]:
define(['child', 'cat'])

child, cat
GPT-3:
[('cat', 0.947524137868798), ('child', 0.9475241335621452), ('kid', 0.9180447810047716), ('baby', 0.9138591242021441), ('kitten', 0.9047885104575575), ('dog', 0.9046532322292953), ('pet', 0.8984533967628818), ('animal', 0.8901310545403318), ('boy', 0.8868829640474493), ('children', 0.8822015101718812), ('puppy', 0.8813685314054924), ('person', 0.8803255241804998)]
------------------------------------------------------
OPT:
[('dog', 0.9830002434449965), ('child', 0.9735579216468181), ('worm', 0.9717930721431548), ('monster', 0.9713974979140467), ('bird', 0.9713284430296463), ('kid', 0.969950539483863), ('mother', 0.969501651232742), ('beard', 0.9693399007742011), ('class', 0.9684331589962502), ('cat', 0.9684245172788707), ('skin', 0.9670493342581906), ('rush', 0.9661381858929798)]
------------------------------------------------------
T5:
[('cat', 0.8812374960772453), ('child', 0.8440493139081031), ('pet', 0.7454129293927738), ('baby', 0.698766317621332), ('dog', 0.695

In [95]:
sim_test(['child', 'cat'], 'kitten')

child, cat -> kitten
GPT-3:
0.9047885104575575
------------------------------------------------------
OPT:
0.5442891463487197
------------------------------------------------------
T5:
0.6330784316254306
------------------------------------------------------


In [96]:
define(['young', 'cat'])

young, cat
GPT-3:
[('cat', 0.9331278288688616), ('young', 0.9331278279785762), ('kitten', 0.8996625885040913), ('kid', 0.8901330793283766), ('baby', 0.8891210778937335), ('dog', 0.8885103885806255), ('child', 0.8874221189896552), ('youth', 0.882174272622569), ('pet', 0.8801251442536662), ('animal', 0.8797573008719888), ('girl', 0.876166488060655), ('person', 0.8757114031078514)]
------------------------------------------------------
OPT:
[('young', 0.9751791299621682), ('cat', 0.9731496131090336), ('fat', 0.968856916489618), ('gay', 0.9612320287133607), ('war', 0.9590193295888614), ('tall', 0.9583243746460556), ('hat', 0.9579190274218565), ('rush', 0.9578076953607579), ('our', 0.9573152192666733), ('kid', 0.9561760332511196), ('egg', 0.9561442640044925), ('rich', 0.9557656893382388)]
------------------------------------------------------
T5:
[('young', 0.8563720057536222), ('cat', 0.8544285268353506), ('hot', 0.737655267928665), ('fat', 0.7122579608326018), ('wild', 0.7064684853962617)

In [97]:
sim_test(['young', 'cat'], 'kitten')

young, cat -> kitten
GPT-3:
0.8996625885040913
------------------------------------------------------
OPT:
0.547557284047786
------------------------------------------------------
T5:
0.5679556725914571
------------------------------------------------------


In [98]:
define(['small', 'dog'])

small, dog
GPT-3:
[('dog', 0.9399958761177261), ('small', 0.9399958730833062), ('little', 0.9134433766362262), ('pet', 0.8991715840911413), ('puppy', 0.8927070132559032), ('tiny', 0.8922148496420499), ('animal', 0.8909921065518351), ('short', 0.8838880631620014), ('baby', 0.8822084328741988), ('thing', 0.8749943680950589), ('child', 0.8728807296518869), ('few', 0.8718840034060675)]
------------------------------------------------------
OPT:
[('dog', 0.9831558412261938), ('monster', 0.9827433760115932), ('small', 0.9804882490764347), ('hole', 0.9804754601304237), ('cycle', 0.9784998567996203), ('free', 0.9784584575862563), ('magic', 0.9783679965284131), ('smart', 0.9778510484953217), ('bird', 0.9776071807948667), ('table', 0.9772150082339055), ('bug', 0.9769652659472039), ('venture', 0.9768723544485608)]
------------------------------------------------------
T5:
[('dog', 0.8170603327073246), ('small', 0.7622173343982397), ('little', 0.667222604993311), ('pet', 0.6536806061745689), ('bab

In [99]:
sim_test(['small', 'dog'], 'puppy')

small, dog -> puppy
GPT-3:
0.8927070132559032
------------------------------------------------------
OPT:
0.09475878669905807
------------------------------------------------------
T5:
0.638814109029041
------------------------------------------------------


In [100]:
define(['baby', 'dog'])

baby, dog
GPT-3:
[('dog', 0.9579394486576065), ('baby', 0.9579394474086329), ('puppy', 0.9212896454536426), ('child', 0.9142739688726862), ('animal', 0.9065965439389942), ('pet', 0.9043861499802028), ('kid', 0.8960574466937055), ('boy', 0.8897633912265036), ('infant', 0.8893081838291764), ('cat', 0.8844663929113614), ('kitten', 0.8744299242619464), ('buddy', 0.8740792439600342)]
------------------------------------------------------
OPT:
[('dog', 0.990920407685825), ('monster', 0.990066558968132), ('baby', 0.9898636011040007), ('bird', 0.9801223617050201), ('girl', 0.9788310039534068), ('beard', 0.9786235127244367), ('cycle', 0.9754046259301921), ('bug', 0.974749357104126), ('worm', 0.9742721978479263), ('kid', 0.9730674350614473), ('mother', 0.9728801368264322), ('cream', 0.9724422993325558)]
------------------------------------------------------
T5:
[('dog', 0.9018997256721512), ('baby', 0.8629197034253134), ('pet', 0.7212496992274253), ('kid', 0.7203595612609869), ('child', 0.706147

In [101]:
sim_test(['baby', 'dog'], 'puppy')

baby, dog -> puppy
GPT-3:
0.9212896454536426
------------------------------------------------------
OPT:
0.10269866483198077
------------------------------------------------------
T5:
0.7009589581101818
------------------------------------------------------


In [102]:
define(['child', 'dog'])

child, dog
GPT-3:
[('dog', 0.9570412073458767), ('child', 0.9570412066373551), ('kid', 0.9200777831719624), ('baby', 0.9169293955674677), ('puppy', 0.9130556783394285), ('boy', 0.9035063223908468), ('animal', 0.9029909813496954), ('children', 0.8988634318581056), ('pet', 0.8958156680530311), ('person', 0.879913417818865), ('infant', 0.8793292310479981), ('cat', 0.8767176182317005)]
------------------------------------------------------
OPT:
[('dog', 0.9917105228680183), ('child', 0.990647263476024), ('monster', 0.9776063291480687), ('bird', 0.9764254978185007), ('mother', 0.9756070179123429), ('friend', 0.9733425587778965), ('beard', 0.9719192730835384), ('father', 0.9698470234951111), ('worm', 0.9698162796362099), ('cycle', 0.9691879459612065), ('master', 0.9689023787645084), ('class', 0.9686417409457829)]
------------------------------------------------------
T5:
[('dog', 0.8935171903321618), ('child', 0.8797960040134483), ('children', 0.7410469188392845), ('pet', 0.7281950103047686)

In [103]:
sim_test(['child', 'dog'], 'puppy')

child, dog -> puppy
GPT-3:
0.9130556783394285
------------------------------------------------------
OPT:
0.08940691008402975
------------------------------------------------------
T5:
0.6709082608840916
------------------------------------------------------


In [104]:
define(['young', 'dog'])

young, dog
GPT-3:
[('young', 0.9422530830434187), ('dog', 0.9422530800155708), ('puppy', 0.9045374749167391), ('youth', 0.9006800638381129), ('child', 0.8980646238034686), ('animal', 0.8932896650191718), ('baby', 0.8928591213705375), ('kid', 0.8928500500260077), ('boy', 0.8910397627785615), ('human', 0.8812900601582215), ('male', 0.8807021079274634), ('youngster', 0.8798576486336198)]
------------------------------------------------------
OPT:
[('dog', 0.9813693446966705), ('young', 0.9767914182266759), ('gay', 0.9721638092695848), ('monster', 0.9720860278545316), ('mother', 0.9697216518769816), ('bird', 0.969000604093702), ('rich', 0.9686514287912604), ('star', 0.9677400046934785), ('our', 0.9669429185245526), ('poor', 0.9668350152313224), ('war', 0.9663803383336362), ('worm', 0.9662026623294997)]
------------------------------------------------------
T5:
[('young', 0.8618556093128282), ('dog', 0.8371972601212895), ('boy', 0.7432669491424451), ('dirty', 0.7180279083313046), ('woman', 

In [105]:
sim_test(['young', 'dog'], 'puppy')

young, dog -> puppy
GPT-3:
0.9045374749167391
------------------------------------------------------
OPT:
0.08751207917816747
------------------------------------------------------
T5:
0.6113773039272755
------------------------------------------------------


In [106]:
define(['female', 'fox'])

female, fox
GPT-3:
[('female', 0.9527931996753368), ('woman', 0.9164814346162464), ('girl', 0.9036956970610488), ('lady', 0.8951691992228892), ('person', 0.8842751357784133), ('she', 0.8835343873037842), ('her', 0.8825790484701783), ('model', 0.8795894858655884), ('male', 0.8772117020637068), ('wife', 0.8772104667877371), ('adult', 0.8771900394009434), ('nurse', 0.8767115914074085)]
------------------------------------------------------
OPT:
[('lane', 0.9828524352941527), ('office', 0.9793810069400244), ('female', 0.9780317258388168), ('nation', 0.9766489464474235), ('tube', 0.9763140071161723), ('agent', 0.9756315800360549), ('stroke', 0.9755698975899589), ('usual', 0.9749711543294424), ('male', 0.974871997730114), ('stage', 0.9737267317120807), ('station', 0.9734783785436791), ('successful', 0.9733971334270588)]
------------------------------------------------------
T5:
[('female', 0.8390271763604016), ('male', 0.7177074513352351), ('woman', 0.672015074342644), ('adult', 0.6245378750

In [107]:
sim_test(['female', 'fox'], 'vixen')

female, fox -> vixen
GPT-3:
0.8670901870792083
------------------------------------------------------
OPT:
0.08998460309133026
------------------------------------------------------
T5:
0.2662331920305942
------------------------------------------------------


In [108]:
define(['baby', 'horse'])

baby, horse
GPT-3:
[('horse', 0.9490645024993978), ('baby', 0.949064501538111), ('animal', 0.905346975416798), ('child', 0.9045353755970991), ('dog', 0.891985692776226), ('infant', 0.887446624431285), ('cow', 0.880380170391074), ('kid', 0.8799053589667223), ('boy', 0.8751777239001927), ('puppy', 0.8746243935219384), ('pet', 0.872902679412483), ('bird', 0.8712194225192886)]
------------------------------------------------------
OPT:
[('baby', 0.9845720751784984), ('horse', 0.9832864855667187), ('monster', 0.9798260322765135), ('street', 0.9756301785382034), ('bird', 0.9750210017997741), ('cycle', 0.9749703228285084), ('country', 0.9739155374517527), ('black', 0.9732644907834216), ('dead', 0.9726419923982779), ('key', 0.9724777242237749), ('dog', 0.9713372500472036), ('food', 0.9712979768862947)]
------------------------------------------------------
T5:
[('horse', 0.8925292841765513), ('baby', 0.8197092152727965), ('dog', 0.7419968729293445), ('boy', 0.7267729003485226), ('kid', 0.68172

In [109]:
sim_test(['baby', 'horse'], 'foal')

baby, horse -> foal
GPT-3:
0.898837676393785
------------------------------------------------------
OPT:
0.48001933417654086
------------------------------------------------------
T5:
0.34616138984017464
------------------------------------------------------


In [110]:
define(['young', 'horse'])

young, horse
GPT-3:
[('young', 0.9378399303930005), ('horse', 0.9378399264401631), ('youth', 0.895311848752121), ('animal', 0.8876493716468311), ('child', 0.8837835211016044), ('baby', 0.8790142930313476), ('human', 0.8746101702719573), ('kid', 0.872226541069931), ('boy', 0.8720524651410909), ('youngster', 0.8716034787074953), ('person', 0.8715474867573196), ('dog', 0.8708787240182073)]
------------------------------------------------------
OPT:
[('horse', 0.9749048882541946), ('young', 0.974171407336037), ('dead', 0.9683135547139066), ('country', 0.9678087552479779), ('town', 0.9639366640162028), ('team', 0.962441655314275), ('poor', 0.9619375590985922), ('club', 0.9618484647475039), ('morning', 0.9614615365702204), ('white', 0.9608706493736209), ('bird', 0.9605458049220431), ('black', 0.9602538528219857)]
------------------------------------------------------
T5:
[('horse', 0.8408128950201247), ('young', 0.8384695980006518), ('boy', 0.7579258416456549), ('boys', 0.7163240780703627), 

In [111]:
sim_test(['young', 'horse'], 'foal')

young, horse -> foal
GPT-3:
0.8866399443126901
------------------------------------------------------
OPT:
0.4529820072795926
------------------------------------------------------
T5:
0.3132413281375827
------------------------------------------------------


In [112]:
define(['female', 'foal'])

female, foal
GPT-3:
[('female', 0.9414426110957205), ('woman', 0.9023398225394237), ('girl', 0.8995833035266989), ('baby', 0.8882563354479935), ('daughter', 0.8847205368748785), ('lady', 0.8840998519805066), ('horse', 0.8805419403732151), ('puppy', 0.8798127396870135), ('child', 0.8792504370847052), ('her', 0.8751274201916007), ('animal', 0.8711305915038804), ('person', 0.8705972795999369)]
------------------------------------------------------
OPT:
[('female', 0.9458197662128239), ('male', 0.9447835445551779), ('knife', 0.9415064874249001), ('blade', 0.9377621618629324), ('cloth', 0.936284720426592), ('adult', 0.9359597124236129), ('planet', 0.935060274811107), ('monster', 0.9344531592375602), ('inner', 0.933286608509993), ('bone', 0.9330744466155461), ('rider', 0.9330081398703982), ('player', 0.9323248286041619)]
------------------------------------------------------
T5:
[('female', 0.8001152379482673), ('male', 0.7175833457779579), ('woman', 0.6204001843448509), ('adult', 0.60761868

In [113]:
sim_test(['female', 'foal'], 'filly')

female, foal -> filly
GPT-3:
0.8860225310618282
------------------------------------------------------
OPT:
0.6207044491710507
------------------------------------------------------
T5:
0.48173342551550746
------------------------------------------------------


In [114]:
define(['male', 'foal'])

male, foal
GPT-3:
[('male', 0.9388182949345112), ('horse', 0.8872921262811684), ('baby', 0.8871251996481023), ('boy', 0.8849363490020614), ('child', 0.8810628185124674), ('person', 0.8801450713600276), ('guy', 0.8778028635611914), ('puppy', 0.8769869691803249), ('human', 0.876893878333324), ('adult', 0.876662939943205), ('animal', 0.8758439899924589), ('him', 0.8722516571940608)]
------------------------------------------------------
OPT:
[('male', 0.9439315621675876), ('adult', 0.9375842150055317), ('knife', 0.9353082968350666), ('cloth', 0.9328970581413742), ('rider', 0.9328543783487288), ('monster', 0.931941501587186), ('kid', 0.9318808137547028), ('inner', 0.9310841927838434), ('bone', 0.9308323948587791), ('worm', 0.9304077928924285), ('chemist', 0.9287015830689764), ('man', 0.9286522775337375)]
------------------------------------------------------
T5:
[('male', 0.7916795817923703), ('female', 0.6872184991032295), ('mare', 0.5992135542299957), ('muscle', 0.5974196421149989), ('ad

In [115]:
sim_test(['male', 'foal'], 'colt')

male, foal -> colt
GPT-3:
0.8477154522791743
------------------------------------------------------
OPT:
0.5770296433418631
------------------------------------------------------
T5:
0.5506395224855681
------------------------------------------------------


In [116]:
define(['baby', 'male', 'horse'])

baby, male, horse
GPT-3:
[('baby', 0.9387187856239669), ('male', 0.9368728176113901), ('horse', 0.9269243721514857), ('child', 0.9070796001217121), ('animal', 0.905325966310804), ('boy', 0.9047755654029191), ('person', 0.90422308083031), ('adult', 0.9013779914982669), ('human', 0.8993037193648525), ('dog', 0.8983677479248617), ('guy', 0.8919088998378284), ('kid', 0.8899366379948959)]
------------------------------------------------------
OPT:
[('monster', 0.9840507382107972), ('baby', 0.9825750591421272), ('country', 0.9792864501562156), ('male', 0.9781524763729058), ('bird', 0.9776310409742872), ('student', 0.9772837221831294), ('strength', 0.9766604190882386), ('cycle', 0.9760057170324291), ('street', 0.9759872616870052), ('dog', 0.975924804929202), ('worm', 0.9744776341436457), ('pole', 0.973583509775293)]
------------------------------------------------------
T5:
[('horse', 0.8265817870278427), ('male', 0.8235142427077305), ('baby', 0.800974285945569), ('female', 0.754630540323649)

In [117]:
sim_test(['baby', 'male', 'horse'], 'colt')

baby, male, horse -> colt
GPT-3:
0.8583693714665181
------------------------------------------------------
OPT:
0.48557621328560246
------------------------------------------------------
T5:
0.4103208590661196
------------------------------------------------------


In [118]:
define(['young', 'male', 'horse'])

young, male, horse
GPT-3:
[('male', 0.9433378080254695), ('young', 0.9300067632199459), ('horse', 0.9160867199584591), ('person', 0.902200526056268), ('human', 0.9020076440583378), ('boy', 0.8998748256988969), ('youth', 0.8992851886173082), ('adult', 0.8957216812822802), ('guy', 0.8908852000868672), ('animal', 0.8903862178359401), ('child', 0.8901008585214657), ('female', 0.8887492279279591)]
------------------------------------------------------
OPT:
[('male', 0.9789826696487957), ('country', 0.9751932432265508), ('poor', 0.9745365911157031), ('student', 0.9721825678000799), ('team', 0.9701544724283881), ('monster', 0.9697890343216233), ('gay', 0.9697079255014618), ('young', 0.9696210571669417), ('bird', 0.9681092833435644), ('our', 0.9676592493111527), ('white', 0.9674464390310108), ('dog', 0.9671476302859712)]
------------------------------------------------------
T5:
[('male', 0.8383449863873439), ('young', 0.8268722985073331), ('horse', 0.7842718448596588), ('female', 0.7764448480

In [119]:
sim_test(['young', 'male', 'horse'], 'colt')

young, male, horse -> colt
GPT-3:
0.8504949338795591
------------------------------------------------------
OPT:
0.495833882887752
------------------------------------------------------
T5:
0.42422276381417234
------------------------------------------------------


In [120]:
define(['baby', 'female', 'horse'])

baby, female, horse
GPT-3:
[('baby', 0.9412920076078248), ('female', 0.9365718776049596), ('horse', 0.9242000420855035), ('woman', 0.9138946798129771), ('girl', 0.9117734237976436), ('child', 0.9076584258498084), ('animal', 0.9039453553695027), ('person', 0.8996018117046547), ('lady', 0.8965103937078673), ('human', 0.8952461091619439), ('her', 0.8944200805556998), ('dog', 0.8937326730252011)]
------------------------------------------------------
OPT:
[('monster', 0.9843861817864576), ('strength', 0.9829599223412235), ('baby', 0.9812449853180539), ('country', 0.9801375429088839), ('male', 0.9779346810649789), ('student', 0.9772466311346679), ('cycle', 0.9759814682138652), ('white', 0.9753445042962512), ('bird', 0.9751920457401029), ('powerful', 0.9751250575942104), ('wine', 0.9750009231439275), ('magic', 0.9746595005332025)]
------------------------------------------------------
T5:
[('female', 0.8277009553543504), ('horse', 0.8190372154189445), ('baby', 0.7976336706403693), ('woman', 

In [121]:
sim_test(['baby', 'female', 'horse'], 'filly')

baby, female, horse -> filly
GPT-3:
0.859647641111861
------------------------------------------------------
OPT:
0.5053087376352451
------------------------------------------------------
T5:
0.359838835098246
------------------------------------------------------


In [122]:
define(['young', 'female', 'horse'])

young, female, horse
GPT-3:
[('female', 0.940207873431395), ('young', 0.9297459778224083), ('horse', 0.9147500954343), ('woman', 0.9127093836726465), ('girl', 0.9120263698916248), ('human', 0.8993100104264797), ('person', 0.8989352485648829), ('youth', 0.8975073374261869), ('male', 0.8949893359451672), ('model', 0.8928506359063189), ('baby', 0.8923257103860075), ('child', 0.8920470894344714)]
------------------------------------------------------
OPT:
[('male', 0.9790778954822426), ('country', 0.9764478192736391), ('poor', 0.9751500100443073), ('female', 0.9739404967921073), ('strength', 0.973887737956854), ('team', 0.972709811546673), ('student', 0.972540507632543), ('powerful', 0.9721480023560838), ('monster', 0.9706511086746167), ('usual', 0.9704021217426404), ('white', 0.9701466430967177), ('less', 0.9698541971440138)]
------------------------------------------------------
T5:
[('female', 0.8429649090225332), ('young', 0.8260557247515921), ('male', 0.7871466639391961), ('woman', 0.

In [123]:
sim_test(['young', 'female', 'horse'], 'filly')

young, female, horse -> filly
GPT-3:
0.8607585044963459
------------------------------------------------------
OPT:
0.5058479322823306
------------------------------------------------------
T5:
0.35535048558920523
------------------------------------------------------


In [124]:
define(['adult', 'female', 'horse'])

adult, female, horse
GPT-3:
[('adult', 0.9444294981433565), ('female', 0.9416866783507857), ('woman', 0.9218670746309116), ('horse', 0.9206743421205796), ('person', 0.9124634754335659), ('human', 0.9066807420552088), ('male', 0.905663355887636), ('mature', 0.9022077744053907), ('animal', 0.9000510385027072), ('girl', 0.8993813559475922), ('lady', 0.8986021848994239), ('gender', 0.8917480143675189)]
------------------------------------------------------
OPT:
[('student', 0.9852222683503775), ('adult', 0.9850100580890869), ('male', 0.9843745122091689), ('country', 0.9808249652741391), ('agent', 0.980635082410085), ('monster', 0.9800593848319423), ('female', 0.9794704845289381), ('strength', 0.9774731856048023), ('nation', 0.9767853104685384), ('enemy', 0.9764880386470526), ('usual', 0.9757410637681739), ('powerful', 0.9756682755185682)]
------------------------------------------------------
T5:
[('female', 0.8672746867433117), ('adult', 0.8454493730629273), ('male', 0.8090864788077389), 

In [125]:
sim_test(['adult', 'female', 'horse'], 'mare')

adult, female, horse -> mare
GPT-3:
0.8542087504943072
------------------------------------------------------
OPT:
0.9347709356872019
------------------------------------------------------
T5:
0.6465928557262918
------------------------------------------------------


In [126]:
define(['adult', 'male', 'pig'])

adult, male, pig
GPT-3:
[('adult', 0.9487020979483003), ('male', 0.9483908870254072), ('person', 0.910874200580874), ('pig', 0.9106606571649682), ('human', 0.9069111086052497), ('man', 0.898885049006229), ('mature', 0.8958649538802939), ('guy', 0.8934540427941189), ('animal', 0.8911149824050508), ('user', 0.8889063620142588), ('parent', 0.8883032100813684), ('dog', 0.8870820488197197)]
------------------------------------------------------
OPT:
[('adult', 0.9717457795796076), ('male', 0.9643878784450278), ('student', 0.9605697073646685), ('agent', 0.95594293864721), ('animal', 0.9531404858198561), ('monster', 0.9524783963559503), ('egg', 0.9509656483931439), ('female', 0.950878927331565), ('individual', 0.94947326946254), ('dog', 0.9479449271215548), ('worm', 0.9475169988018881), ('usual', 0.9474572162207531)]
------------------------------------------------------
T5:
[('adult', 0.8642177943437807), ('male', 0.8513984377591179), ('female', 0.7832017642284079), ('infant', 0.709265861757

In [127]:
sim_test(['adult', 'male', 'pig'], 'boar')

adult, male, pig -> boar
GPT-3:
0.8856531564489787
------------------------------------------------------
OPT:
0.6300964469161798
------------------------------------------------------
T5:
0.28476485762938675
------------------------------------------------------


In [128]:
define(['adult', 'male', 'human'])

adult, male, human
GPT-3:
[('male', 0.959952309741523), ('adult', 0.9594236427285727), ('human', 0.9562139328700605), ('person', 0.9329763817668865), ('man', 0.907948985586051), ('individual', 0.9074837296300489), ('regular', 0.9006497680321661), ('female', 0.8970765597533072), ('guy', 0.896827886234498), ('mature', 0.8957991369863132), ('gender', 0.8956374270801458), ('age', 0.8924075819783439)]
------------------------------------------------------
OPT:
[('male', 0.9870355244791095), ('adult', 0.9861067364922234), ('animal', 0.9853876806966206), ('traditional', 0.9784527833800645), ('functional', 0.9772362817234472), ('universal', 0.9756164966033168), ('female', 0.974325616850968), ('capitalist', 0.9735305807722622), ('verbal', 0.9732154150822101), ('active', 0.9727996880121745), ('oral', 0.972166435209249), ('student', 0.9717860005930906)]
------------------------------------------------------
T5:
[('adult', 0.8639132235001863), ('male', 0.8467026033911327), ('human', 0.818754455474

In [129]:
sim_test(['adult', 'male', 'human'], 'man')

adult, male, human -> man
GPT-3:
0.907948985586051
------------------------------------------------------
OPT:
0.9384335730367203
------------------------------------------------------
T5:
0.6464197622914151
------------------------------------------------------


In [130]:
define(['adult', 'female', 'human'])

adult, female, human
GPT-3:
[('human', 0.9569578620371786), ('adult', 0.9538376873318073), ('female', 0.9502529956021907), ('person', 0.9330493904359031), ('woman', 0.931666599516325), ('male', 0.9162142639353046), ('individual', 0.9073796526395572), ('regular', 0.9011840744270789), ('gender', 0.9005836992011926), ('mature', 0.9001234343229937), ('girl', 0.8982965809744285), ('lady', 0.8957944418920751)]
------------------------------------------------------
OPT:
[('male', 0.9867318022712125), ('adult', 0.9845548031128055), ('animal', 0.9828830410603928), ('female', 0.9819539152348169), ('traditional', 0.9817620447349067), ('functional', 0.9804870480843118), ('universal', 0.9779142879005508), ('active', 0.977188079618499), ('capitalist', 0.9763887800414315), ('powerful', 0.9759723976744274), ('aggressive', 0.975613812928612), ('consumer', 0.9751111979978855)]
------------------------------------------------------
T5:
[('adult', 0.8658371092247303), ('female', 0.8522225336294019), ('hum

In [131]:
sim_test(['adult', 'female', 'human'], 'woman')

adult, female, human -> woman
GPT-3:
0.931666599516325
------------------------------------------------------
OPT:
0.9385508787294747
------------------------------------------------------
T5:
0.710133218138796
------------------------------------------------------


### D - Philosophy, math, miscellaneous
This section is small now - it may move soon.

In [132]:
define(['true', 'belief'])

true, belief
GPT-3:
[('true', 0.945101777514645), ('belief', 0.945101775977824), ('believe', 0.9314666865404644), ('truth', 0.917202119882703), ('faith', 0.903768888120221), ('actual', 0.8976471909094058), ('real', 0.8906441948916156), ('according', 0.8894279043420547), ('trust', 0.8889554635894935), ('genuine', 0.8887203363063545), ('reason', 0.8853575499136224), ('meaning', 0.8849147218268709)]
------------------------------------------------------
OPT:
[('true', 0.889819615596526), ('wrong', 0.8762317350186533), ('racist', 0.8761940024488432), ('crazy', 0.8757129411617981), ('false', 0.8709278862421237), ('interesting', 0.8680786039648204), ('weak', 0.8671028507388212), ('bold', 0.866507987527807), ('hate', 0.8653618193757283), ('wise', 0.865110374621999), ('faith', 0.8634330312494872), ('real', 0.8631752509397776)]
------------------------------------------------------
T5:
[('belief', 0.8902315632357154), ('true', 0.7689130980519974), ('believe', 0.7360326960165493), ('faith', 0.71

In [133]:
sim_test(['true', 'belief'], 'knowledge')

true, belief -> knowledge
GPT-3:
0.8552819148186207
------------------------------------------------------
OPT:
0.8140756052998738
------------------------------------------------------
T5:
0.6432610020493008
------------------------------------------------------


## Define Helpers (for magnitude)

In [134]:
# this function doesn't allow model specification, as GPT-3 vectors are normalized.
def sum_complexity(word, vec=False, model='opt'):
    """
    Args:
        word: iterable, string, or vector   -> embedding to be summed.
        vec: bool                           -> True if word is already vector.
        model: str                          -> 'opt' or 't5'. No gpt weights.
    Returns:
        Float score of the sum of the absolute value of each datapoint in the embedding.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model=model)
        else:
            phrase = positive(word, model=model)
    
    modified = [abs(float(x)) for x in phrase[0]]
    return sum(modified)
    

In [135]:
def above_zero_complexity(word, vec=False, threshold=0.3, model='opt'):
    """
    Args:
        word: iterable, string, or vector   -> embedding to be evaluated
        vec: bool                           -> True if word is already vector.
        threshold: float                    -> Absolute value cutoff point
        model: str                          -> 'opt' or 't5'. No gpt weights.
    Returns:
        Float score of the amount of 'signifigant' features according to the threshold,
        divided by the number of total dimensions.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model=model)
        else:
            phrase = positive(word, model=model)

    modified = [abs(float(x)) for x in phrase[0] if abs(float(x)) > threshold]
    return len(modified) / phrase.shape[1]

In [136]:
def compare_complexity(words, threshold=0.3):
    """
    Args:
        words: iterable     -> items to be converted to tokens and compared.
        threshold: fload    -> threshold for signifigant features with a_z_c()
    Returns:
        None. Formatted printing results for each item in words.
    """
    for w in words:
        print(w)
        print("OPT-13b:")
        print(f"Sum Complexity: {sum_complexity(w)}")
        print(f"Above Zero Complexity t-[{threshold}]: {above_zero_complexity(w, threshold=threshold)}")
        print('--------------------------------------------------')
        print("T5:")
        print(f"Sum Complexity: {sum_complexity(w, model='t5')}")
        print(f"Above Zero Complexity t-[{threshold}]: {above_zero_complexity(w, threshold=threshold, model='t5')}")
        print('--------------------------------------------------')
        print()

## Conduct Magnitude Tests
This is a shorter section, and all of our tests are on OPT-1.3b.

In [137]:
compare_complexity(['man', 'bachelor', 'husband', 'father'])

man
OPT-1.3b:
Sum Complexity: 4560.874884605697
Above Zero Complexity t-[0.3]: 0.7720703125
--------------------------------------------------
T5:
Sum Complexity: 75.15917153300698
Above Zero Complexity t-[0.3]: 0.0029296875
--------------------------------------------------

bachelor
OPT-1.3b:
Sum Complexity: 5267.261508663384
Above Zero Complexity t-[0.3]: 0.8142578125
--------------------------------------------------
T5:
Sum Complexity: 76.31267658150999
Above Zero Complexity t-[0.3]: 0.0029296875
--------------------------------------------------

husband
OPT-1.3b:
Sum Complexity: 5648.918998071234
Above Zero Complexity t-[0.3]: 0.8083984375
--------------------------------------------------
T5:
Sum Complexity: 74.88139465603
Above Zero Complexity t-[0.3]: 0.0
--------------------------------------------------

father
OPT-1.3b:
Sum Complexity: 5456.695524385167
Above Zero Complexity t-[0.3]: 0.81171875
--------------------------------------------------
T5:
Sum Complexity: 71.54839

In [138]:
compare_complexity(['woman', 'wife', 'mother'])

woman
OPT-1.3b:
Sum Complexity: 5267.089455174949
Above Zero Complexity t-[0.3]: 0.794140625
--------------------------------------------------
T5:
Sum Complexity: 68.76266775527999
Above Zero Complexity t-[0.3]: 0.0009765625
--------------------------------------------------

wife
OPT-1.3b:
Sum Complexity: 5315.254698051018
Above Zero Complexity t-[0.3]: 0.796484375
--------------------------------------------------
T5:
Sum Complexity: 71.48278228784298
Above Zero Complexity t-[0.3]: 0.0009765625
--------------------------------------------------

mother
OPT-1.3b:
Sum Complexity: 5245.847844181448
Above Zero Complexity t-[0.3]: 0.7970703125
--------------------------------------------------
T5:
Sum Complexity: 74.65946013634004
Above Zero Complexity t-[0.3]: 0.001953125
--------------------------------------------------



In [139]:
compare_complexity(['cat', 'kitten'])

cat
OPT-1.3b:
Sum Complexity: 4594.572631555979
Above Zero Complexity t-[0.3]: 0.76796875
--------------------------------------------------
T5:
Sum Complexity: 80.59261954760503
Above Zero Complexity t-[0.3]: 0.005859375
--------------------------------------------------

kitten
OPT-1.3b:
Sum Complexity: 5325.624572697723
Above Zero Complexity t-[0.3]: 0.818359375
--------------------------------------------------
T5:
Sum Complexity: 75.22567567549004
Above Zero Complexity t-[0.3]: 0.00390625
--------------------------------------------------



In [140]:
compare_complexity(['dog', 'puppy'])

dog
OPT-1.3b:
Sum Complexity: 5219.776013053286
Above Zero Complexity t-[0.3]: 0.8005859375
--------------------------------------------------
T5:
Sum Complexity: 75.50303368280998
Above Zero Complexity t-[0.3]: 0.0009765625
--------------------------------------------------

puppy
OPT-1.3b:
Sum Complexity: 5734.2633102057325
Above Zero Complexity t-[0.3]: 0.8271484375
--------------------------------------------------
T5:
Sum Complexity: 70.79011017358782
Above Zero Complexity t-[0.3]: 0.0009765625
--------------------------------------------------



In [141]:
compare_complexity(['duck', 'duckling'])

duck
OPT-1.3b:
Sum Complexity: 5327.827787368452
Above Zero Complexity t-[0.3]: 0.814453125
--------------------------------------------------
T5:
Sum Complexity: 83.1197584920601
Above Zero Complexity t-[0.3]: 0.0068359375
--------------------------------------------------

duckling
OPT-1.3b:
Sum Complexity: 5995.632258473692
Above Zero Complexity t-[0.3]: 0.846484375
--------------------------------------------------
T5:
Sum Complexity: 76.75135235866998
Above Zero Complexity t-[0.3]: 0.009765625
--------------------------------------------------



In [142]:
compare_complexity(['set', 'powerset'])

set
OPT-1.3b:
Sum Complexity: 5020.276800178536
Above Zero Complexity t-[0.3]: 0.7880859375
--------------------------------------------------
T5:
Sum Complexity: 53.52874527122802
Above Zero Complexity t-[0.3]: 0.0
--------------------------------------------------

powerset
OPT-1.3b:
Sum Complexity: 6133.837665715833
Above Zero Complexity t-[0.3]: 0.8423828125
--------------------------------------------------
T5:
Sum Complexity: 78.73620877492694
Above Zero Complexity t-[0.3]: 0.001953125
--------------------------------------------------



In [143]:
compare_complexity(['matrix', 'determinant'])

matrix
OPT-1.3b:
Sum Complexity: 5137.526284368182
Above Zero Complexity t-[0.3]: 0.8064453125
--------------------------------------------------
T5:
Sum Complexity: 91.40458834680985
Above Zero Complexity t-[0.3]: 0.0126953125
--------------------------------------------------

determinant
OPT-1.3b:
Sum Complexity: 5833.48253381554
Above Zero Complexity t-[0.3]: 0.8271484375
--------------------------------------------------
T5:
Sum Complexity: 67.44564442279405
Above Zero Complexity t-[0.3]: 0.0
--------------------------------------------------



## Phrase comparison
In the past, we have not had access to models that make a phrase, so we use positives. I will explore some of those combinations below.
Phrases of interest:
* "white house"
* "baby duck"
* "logical belief"

and so on...

In [144]:
define(['logical belief']) # this usage should work.

logical belief
GPT-3:
[('logical', 0.90632635529395), ('logic', 0.875410470078137), ('rational', 0.8409036068990172), ('belief', 0.8356976843828251), ('rationals', 0.8340560236875274), ('reasoning', 0.8318158373723321), ('understanding', 0.8193263779230331), ('believe', 0.8178878774738392), ('credibility', 0.8167356105198776), ('justification', 0.8160767763353183), ('mathematical', 0.8124806874202757), ('philosophical', 0.8057238954089894)]
------------------------------------------------------
OPT:
[('logical', 0.7816086781011855), ('logic', 0.7059903204496798), ('consciousness', 0.7029631747515975), ('reasoning', 0.6853467571315791), ('ideology', 0.6839891380101328), ('characterize', 0.677803939081622), ('characteristic', 0.6760224775753598), ('belief', 0.6746104232437631), ('methodology', 0.6709154181373368), ('interpretation', 0.6705476984431957), ('irrationals', 0.667330101441739), ('ridiculous', 0.66362206865118)]
------------------------------------------------------
T5:
[('logi

In [145]:
sim_test(["white", "house"], 'white house') # this is also valid!

white, house -> white house
GPT-3:
0.8811608112517997
------------------------------------------------------
OPT:
0.5891347180204866
------------------------------------------------------
T5:
0.4876257179694703
------------------------------------------------------


In [146]:
sim_test(['white house'], 'Joe Biden') # we can even do unique, new tests with our wrappers!

white house -> Joe Biden
GPT-3:
0.7825392090482628
------------------------------------------------------
OPT:
0.5587546
------------------------------------------------------
T5:
0.33354205
------------------------------------------------------


In [147]:
define(['baby duck'])

baby duck
GPT-3:
[('duckling', 0.9265918639547074), ('duck', 0.8781261407617771), ('baby', 0.840823223679302), ('chicken', 0.8302024124326883), ('feather', 0.8072321043118504), ('kitten', 0.8069924266209726), ('puppy', 0.8051538990362775), ('infant', 0.8005145471349074), ('bird', 0.7997014350912526), ('cute', 0.7969249627229725), ('sheep', 0.7934043151335479), ('youngster', 0.7827481721332368)]
------------------------------------------------------
OPT:
[('firefighter', 0.7314592271783955), ('baby', 0.7175295254836783), ('cocktail', 0.7125646752361026), ('firework', 0.7105109974304942), ('boyfriend', 0.7095790960476462), ('youngster', 0.704863867220438), ('meat', 0.6999309220520096), ('worthwhile', 0.6962713818203949), ('handful', 0.6934663084169188), ('kidney', 0.6925797363795848), ('brothers', 0.6907755261457802), ('neck', 0.6889934486988692)]
------------------------------------------------------
T5:
[('duck', 0.5598220834662264), ('puppy', 0.5229867836091815), ('youngster', 0.52162

In [148]:
sim_test(['baby', 'duck'], 'baby duck')

baby, duck -> baby duck
GPT-3:
0.9071863942339433
------------------------------------------------------
OPT:
0.8216294947602424
------------------------------------------------------
T5:
0.6304577533317495
------------------------------------------------------


In [149]:
sim_test(['baby duck'], 'duckling')

baby duck -> duckling
GPT-3:
0.9265918639547074
------------------------------------------------------
OPT:
0.5709158877156502
------------------------------------------------------
T5:
0.4698634140376625
------------------------------------------------------


## Phrase Comparison
This is all probably useless, depending on your interpretation of the tokenization -> embedding process in relation to preservation of overall idea.

In [150]:
def phrase_comparison(phrases):
    """
    Args:
        phrases: iterable   -> iterable of novel phrases to compare.
    Returns:
        None. Formats and facilitates phrase comparison of an iterable of novel phrases.
    """
    enc = [(positive([str(x)], 'gpt'), positive([str(x)], 'opt'), positive([str(x)], 't5')) for x in phrases]
    print(*phrases, sep=', ')
    for i in range(len(phrases)):
        for j in range(i+1, len(phrases)):
            print(f"{phrases[i]} -> {phrases[j]}")
            print(f"GPT-3: {cosine_similarity(enc[i][0], enc[j][0])[0][0]}")
            print(f"OPT-1.3B: {cosine_similarity(enc[i][1], enc[j][1])[0][0]}")
            print(f"T5: {cosine_similarity(enc[i][2], enc[j][2])[0][0]}")
            print('------------------------------------------------------')
    print("\nPhrase Definitions:")
    for p in phrases:
        define([p])

### A - spouse phrases

In [151]:
phrase_comparison(['unmarried man', 'a bachelor is an unmarried man'])

unmarried man, a bachelor is an unmarried man
unmarried man -> a bachelor is an unmarried man
GPT-3: 0.8843229523445623
OPT-1.3B: 0.7220609188079834
T5: 0.5093932151794434
------------------------------------------------------

Phrase Definitions:
unmarried man
GPT-3:
[('unmarried', 0.9038285094068708), ('unemployed', 0.8324932215599423), ('bachelor', 0.8306343231477202), ('gentleman', 0.8281874426470867), ('businessman', 0.8210609185091445), ('male', 0.8201344480908059), ('single', 0.81489249198641), ('alone', 0.8142876500176159), ('husband', 0.8091042844519427), ('bachelors', 0.8081341558526824), ('divorced', 0.8076394088326454), ('man', 0.8053249700215654)]
------------------------------------------------------
OPT:
[('unmarried', 0.8460898844808522), ('unemployed', 0.7205349175337545), ('entertainer', 0.7039803579229444), ('unwillingly', 0.7035865496662271), ('novelist', 0.7009607287948922), ('meantime', 0.6983668537169969), ('allegedly', 0.6976766112184738), ('dishonestly', 0.6966

In [152]:
phrase_comparison(['adult female horse', 'Look, a mare!'])

adult female horse, Look, a mare!
adult female horse -> Look, a mare!
GPT-3: 0.8452115583487287
OPT-1.3B: 0.5291064977645874
T5: 0.31007763743400574
------------------------------------------------------

Phrase Definitions:
adult female horse
GPT-3:
[('horse', 0.8288033822944456), ('female', 0.7964569545628017), ('princess', 0.7771064693330306), ('woman', 0.7707933317763727), ('stall', 0.7673353792626918), ('adult', 0.7610707342610956), ('grandmother', 0.7604472186774185), ('herself', 0.760441908702997), ('granddaughter', 0.7554479094342272), ('sheep', 0.7549275385773628), ('lady', 0.7521306068722473), ('queen', 0.751501047868335)]
------------------------------------------------------
OPT:
[('lesbian', 0.6667401224487141), ('businessman', 0.6646696087564846), ('characterize', 0.6512598835046428), ('fashionable', 0.6506953977466368), ('boyfriend', 0.6484467304470836), ('herself', 0.6469788686781592), ('hairdresser', 0.6355630280837509), ('ourselves', 0.6343375458201232), ('musician', 

In [153]:
phrase_comparison(['What do you call an unmarried man?', 'What do you call a married man?'])

What do you call an unmarried man?, What do you call a married man?
What do you call an unmarried man? -> What do you call a married man?
GPT-3: 0.8787195750944905
OPT-1.3B: 0.9781855940818787
T5: 0.8036864995956421
------------------------------------------------------

Phrase Definitions:
What do you call an unmarried man?
GPT-3:
[('unmarried', 0.7945296472674697), ('unemployed', 0.7585959209681947), ('gentleman', 0.7424972354566222), ('bachelor', 0.7351107186608479), ('single', 0.7273255418628072), ('businessman', 0.7259265068432967), ('useless', 0.7226069654841891), ('male', 0.7183357626669026), ('blank', 0.7166851928527622), ('himself', 0.7165732910435403), ('divorced', 0.7157649718732171), ('alone', 0.7156303886765757)]
------------------------------------------------------
OPT:
[('i.e.', 0.764717172141247), ('supposedly', 0.7313449524504125), ('e.g.', 0.7294576495941798), ('apparently', 0.7178310502805796), ('meantime', 0.704550086499821), ('namely', 0.6894467206832874), ('hopef

In [154]:
phrase_comparison(['My dog had babies!', 'Do you want to play with a puppy?'])

My dog had babies!, Do you want to play with a puppy?
My dog had babies! -> Do you want to play with a puppy?
GPT-3: 0.795130930915763
OPT-1.3B: 0.7033061385154724
T5: 0.45165348052978516
------------------------------------------------------

Phrase Definitions:
My dog had babies!
GPT-3:
[('puppy', 0.767713874678072), ('pregnancy', 0.7456566656317866), ('reproduction', 0.7455586263140876), ('congratulation', 0.7368566227313866), ('sheep', 0.7288713893909373), ('adoption', 0.7278466244900033), ('celebration', 0.727587661419159), ('breed', 0.725530170054161), ('fabulous', 0.7224545380334513), ('dog', 0.7198224447916577), ('conception', 0.7197008991485343), ('pregnant', 0.7191196449207902)]
------------------------------------------------------
OPT:
[('i.e.', 0.6726530081449369), ('supposedly', 0.6487115257815942), ('apparently', 0.647119743624026), ('e.g.', 0.6426301305705664), ('meantime', 0.6313687355702751), ('hopefully', 0.606903338925777), ('basically', 0.6015607771527731), ('alleg

In [155]:
phrase_comparison([
    'Knowledge is a true belief.', 
    'An argument is a logical belief.',
    'A spiritual belief is religion.'
])

Knowledge is a true belief., An argument is a logical belief., A spiritual belief is religion.
Knowledge is a true belief. -> An argument is a logical belief.
GPT-3: 0.8140347277983235
OPT-1.3B: 0.7022847533226013
T5: 0.6721267700195312
------------------------------------------------------
Knowledge is a true belief. -> A spiritual belief is religion.
GPT-3: 0.8285514797266937
OPT-1.3B: 0.7650288343429565
T5: 0.5165912508964539
------------------------------------------------------
An argument is a logical belief. -> A spiritual belief is religion.
GPT-3: 0.8104783653492047
OPT-1.3B: 0.7706027030944824
T5: 0.5986865162849426
------------------------------------------------------

Phrase Definitions:
Knowledge is a true belief.
GPT-3:
[('knowledge', 0.7949999136488028), ('belief', 0.758882444295396), ('certainty', 0.7541858887209703), ('realization', 0.753050039730655), ('confirmation', 0.7493754446796699), ('understanding', 0.7487918348206573), ('credibility', 0.747152831065375), ('au

## Average Similarity
What is the average similarity between words in our dictionary for each model?

In [18]:
def make_avg_tbl(model='opt', extra_vocab=[]):
    """
    Args:
        model: str  -> 'gpt', 'opt', or 't5'
        extra_vocab: iterable of extra strings to be processed like vocab
    Returns:
        Table that contains the average cosine similarity of each word in our vocabulary
        to each other word, possibly including user supplied extra vocab.
    """
    print(f"Making average table for model: {model}")
    # list comprehension is O(n) where n is vocab length - 5124
    tbl_vocab = [str(x) for x in vocab + list(extra_vocab)]
    # static lookup O(1)
    tbl_sz = len(tbl_vocab)
    # dictionary comprehension - O(n)
    table = {k: 0.0 for k in tbl_vocab}
    # total so far - O(2n)

    # O(n^2)
    for term_a in tbl_vocab: # O(n)  * 
        a = positive([term_a], model)
        for term_b in tbl_vocab: # O(n)
            if term_a != term_b:
                b = positive([term_b], model)
                table[term_a] += cosine_similarity(a, b)[0][0]
        table[term_a] /= tbl_sz
    print('done')
    return table                

In [19]:
def calc_avg_sim():
    # Does this need a header? Wrapper.
    print("What is our average connection strength?")
    gpt_tbl = make_avg_tbl(model='gpt')
    t5_tbl = make_avg_tbl(model='t5')
    opt_tbl = make_avg_tbl()
    sz = len(opt_tbl)
    print("GPT-3:")
    print(sum(gpt_tbl.values()) / sz)
    print('------------------------------------------------------')
    print('OPT-13B:')
    print(sum(opt_tbl.values()) / sz)
    print('------------------------------------------------------')
    print('T5:')
    print(sum(t5_tbl.values()) / sz)
    print('------------------------------------------------------')

In [20]:
calc_avg_sim()
# takes ~500m

What is our average connection strength?
Making average table for model: gpt
done
Making average table for model: t5
done
Making average table for model: opt
done
GPT-3:
0.752081127276011
------------------------------------------------------
OPT-13B:
0.41978558431995905
------------------------------------------------------
T5:
0.3472237501321694
------------------------------------------------------
